# Ordering by metrics and retraining phase

## Dataset: MNIST

## Experiment configuration 2


In [1]:
import argparse

import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

import matplotlib.pyplot as plt

#from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.regularizers import l2

import glob
import os
import cv2
import tensorflow.keras as keras


import time
import argparse

from tqdm import tqdm

from tensorflow.keras.models import load_model, Model


In [2]:
tf.__version__ == '2.5.0' # Version of tensorflow

True

In [3]:
cd ../utils/

D:\guided-retraining\utils


In [4]:
# utils for project
import utils_guided_retraining2 as utils

2.5.0
keras
2.5.0


In [5]:
#cd '../notebooks/'

In [6]:
dataset = "fashion"

In [7]:
x_train,y_train = utils.get_data(dataset,"Train",True)
x_val,y_val = utils.get_data(dataset,"Val",True)
x_test,y_test = utils.get_data(dataset,"Test",True)

D:/guided-retraining/data/fashion/x_train.npy
x_set len:  48999
D:/guided-retraining/data/fashion/y_train.npy
y_set len:  48999
D:/guided-retraining/data/fashion/x_val.npy
x_set len:  14000
D:/guided-retraining/data/fashion/y_val.npy
y_set len:  14000
D:/guided-retraining/data/fashion/x_test.npy
x_set len:  7001
D:/guided-retraining/data/fashion/y_test.npy
y_set len:  7001


In [8]:
x_train_and_adversary,y_train_and_adversary = utils.get_data(dataset,"Train_and_adversary",True)

D:/guided-retraining/data/data_adversarial_july/fashion/train_and_adversary.npy
x_set len:  55998
D:/guided-retraining/data/data_adversarial_july/fashion/train_and_adversary_labels.npy
y_set len:  55998


In [9]:
x_adversary_training = x_train_and_adversary[len(x_train):]
print(len(x_adversary_training))
y_adversary_training = y_train_and_adversary[len(y_train):]

print(len(y_adversary_training))

6999
6999


In [10]:
# Obtaining adversarial examples for testing 
x_test_and_adversary,y_test_and_adversary = utils.get_adversarial_data(dataset,'Test_adversarial',True) 

D:/guided-retraining/data/data_adversarial_july/fashion/test_and_adversary.npy D:/guided-retraining/data/data_adversarial_july/fashion/test_and_adversary_labels.npy
D:/guided-retraining/data/data_adversarial_july/fashion/test_and_adversary.npy
x_set len:  14000
D:/guided-retraining/data/data_adversarial_july/fashion/test_and_adversary_labels.npy
y_set len:  14000


In [11]:
x_adversary_test = x_test_and_adversary[len(x_test):]
print(len(x_adversary_test))
y_adversary_test = y_test_and_adversary[len(y_test):]

print(len(y_adversary_test))

6999
6999


In [12]:
#del x_train, x_test, x_val
#del x_test_and_adversary, y_test_and_adversary
#del x_adversary_test,y_adversary_test

In [12]:
import gc
gc.collect()

110

## ----

In [13]:
# Original model 

model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/"
if(dataset == 'gtsrb'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06"
elif(dataset == 'intel'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/intel_model_21_10"
elif(dataset == 'mnist'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/models2"
elif(dataset == 'cifar'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/model_02"
elif(dataset == 'fashion'):
    model_dir = "D:/guided-retraining/models/model_fashion_2"


In [14]:
print(model_dir)
model_original = utils.My_model(dataset,True, model_dir)

D:/guided-retraining/models/model_fashion_2
Model loaded correctly


In [15]:
print(x_train_and_adversary.shape)
len(x_train_and_adversary)//20

(55998, 28, 28, 1)


2799

In [16]:
n_data_points = 2800

## Loading LSA and DSA values 

In [18]:
lsa_direction = "D:/guided-retraining/data/"+dataset+"/lsa_values.npy"

In [19]:
lsa_values = np.load(lsa_direction)


In [20]:

# Obtaining top n images by LSA values
top_images_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),x_train_and_adversary)
top_labels_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),y_train_and_adversary)


In [21]:
m = n_data_points
n = 0
image_sets_lsa = []
label_sets_lsa = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_lsa)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_lsa)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_lsa)%m))
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m])
    image_sets_lsa.append(top_images_by_lsa_n)
    label_sets_lsa.append(top_labels_by_lsa_n)
    print(len(top_images_by_lsa_n))
    n += m

0 :
0  ->  2800
2800
1 :
0  ->  5600
5600
2 :
0  ->  8400
8400
3 :
0  ->  11200
11200
4 :
0  ->  14000
14000
5 :
0  ->  16800
16800
6 :
0  ->  19600
19600
7 :
0  ->  22400
22400
8 :
0  ->  25200
25200
9 :
0  ->  28000
28000
10 :
0  ->  30800
30800
11 :
0  ->  33600
33600
12 :
0  ->  36400
36400
13 :
0  ->  39200
39200
14 :
0  ->  42000
42000
15 :
0  ->  44800
44800
16 :
0  ->  47600
47600
17 :
0  ->  50400
50400
18 :
0  ->  53200
53200
19 :
Last
0  ->  58798
55998


In [22]:
len(image_sets_lsa[-1])

55998

## Training guided by LSA

In [23]:
len(label_sets_lsa)

20

In [24]:
print(model_dir)

models_lsa = []
for i in range(len(label_sets_lsa)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_lsa.append(model)


D:/guided-retraining/models/model_fashion_2
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [25]:
models_lsa[-1].evaluate(x_test,y_test)
models_lsa[0].evaluate(x_test,y_test)

219/219 [==============================] - 2s 8ms/step - loss: 0.2254 - accuracy: 0.9190


[0.2253975123167038, 0.9190115928649902]

In [26]:
models_lsa[1].evaluate(x_test_and_adversary,y_test_and_adversary)

438/438 [==============================] - 4s 8ms/step - loss: 0.5032 - accuracy: 0.8342


[0.5032416582107544, 0.8342142701148987]

In [27]:
models_lsa[0].evaluate(x_test_and_adversary,y_test_and_adversary)

438/438 [==============================] - 3s 8ms/step - loss: 0.5032 - accuracy: 0.8342


[0.5032416582107544, 0.8342142701148987]

In [28]:
n = 0
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n=n+1

Epoch 1/10
44/44 [==============================] - 6s 115ms/step - loss: 0.4617 - accuracy: 0.8643 - val_loss: 0.2832 - val_accuracy: 0.8894
Epoch 2/10
44/44 [==============================] - 5s 119ms/step - loss: 0.3073 - accuracy: 0.8982 - val_loss: 0.2828 - val_accuracy: 0.8910
Epoch 3/10
44/44 [==============================] - 5s 123ms/step - loss: 0.2417 - accuracy: 0.9189 - val_loss: 0.3025 - val_accuracy: 0.8869
Epoch 4/10
44/44 [==============================] - 5s 121ms/step - loss: 0.2407 - accuracy: 0.9171 - val_loss: 0.3039 - val_accuracy: 0.8846
Epoch 5/10
44/44 [==============================] - 5s 124ms/step - loss: 0.2127 - accuracy: 0.9293 - val_loss: 0.3117 - val_accuracy: 0.8852
Epoch 6/10
44/44 [==============================] - 5s 117ms/step - loss: 0.1928 - accuracy: 0.9307 - val_loss: 0.3125 - val_accuracy: 0.8877
Epoch 7/10
44/44 [==============================] - 5s 117ms/step - loss: 0.1692 - accuracy: 0.9436 - val_loss: 0.3277 - val_accuracy: 0.8862
Epoch 

In [29]:
models_lsa[0].evaluate(x_test_and_adversary,y_test_and_adversary)

438/438 [==============================] - 4s 8ms/step - loss: 0.4196 - accuracy: 0.8683


[0.419632226228714, 0.8682857155799866]

In [30]:
n=1
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

Epoch 1/10
88/88 [==============================] - 8s 85ms/step - loss: 0.2222 - accuracy: 0.9289 - val_loss: 0.2909 - val_accuracy: 0.8912
Epoch 2/10
88/88 [==============================] - 7s 85ms/step - loss: 0.1661 - accuracy: 0.9498 - val_loss: 0.3149 - val_accuracy: 0.8810
Epoch 3/10
88/88 [==============================] - 7s 84ms/step - loss: 0.1414 - accuracy: 0.9582 - val_loss: 0.3135 - val_accuracy: 0.8812
Epoch 4/10
88/88 [==============================] - 8s 88ms/step - loss: 0.1130 - accuracy: 0.9638 - val_loss: 0.3318 - val_accuracy: 0.8803
Epoch 5/10
88/88 [==============================] - 8s 88ms/step - loss: 0.1092 - accuracy: 0.9659 - val_loss: 0.3360 - val_accuracy: 0.8794
Epoch 6/10
88/88 [==============================] - 7s 85ms/step - loss: 0.0896 - accuracy: 0.9716 - val_loss: 0.3527 - val_accuracy: 0.8744
Epoch 7/10
88/88 [==============================] - 7s 85ms/step - loss: 0.0907 - accuracy: 0.9705 - val_loss: 0.3811 - val_accuracy: 0.8734
Epoch 8/10
88

In [31]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

2
Epoch 1/10
132/132 [==============================] - 11s 77ms/step - loss: 0.1984 - accuracy: 0.9379 - val_loss: 0.2898 - val_accuracy: 0.8892
Epoch 2/10
132/132 [==============================] - 10s 76ms/step - loss: 0.1428 - accuracy: 0.9549 - val_loss: 0.3214 - val_accuracy: 0.8772
Epoch 3/10
132/132 [==============================] - 10s 75ms/step - loss: 0.1252 - accuracy: 0.9620 - val_loss: 0.3774 - val_accuracy: 0.8691
Epoch 4/10
132/132 [==============================] - 10s 76ms/step - loss: 0.1191 - accuracy: 0.9645 - val_loss: 0.3437 - val_accuracy: 0.8761
Epoch 5/10
132/132 [==============================] - 10s 76ms/step - loss: 0.1025 - accuracy: 0.9685 - val_loss: 0.3395 - val_accuracy: 0.8783
Epoch 6/10
132/132 [==============================] - 10s 74ms/step - loss: 0.0925 - accuracy: 0.9725 - val_loss: 0.3749 - val_accuracy: 0.8712
Epoch 7/10
132/132 [==============================] - 10s 75ms/step - loss: 0.0850 - accuracy: 0.9717 - val_loss: 0.4060 - val_accurac

In [32]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

3
Epoch 1/10
175/175 [==============================] - 13s 71ms/step - loss: 0.1515 - accuracy: 0.9544 - val_loss: 0.3158 - val_accuracy: 0.8784
Epoch 2/10
175/175 [==============================] - 12s 69ms/step - loss: 0.1177 - accuracy: 0.9654 - val_loss: 0.3295 - val_accuracy: 0.8774
Epoch 3/10
175/175 [==============================] - 12s 70ms/step - loss: 0.1075 - accuracy: 0.9706 - val_loss: 0.3223 - val_accuracy: 0.8809
Epoch 4/10
175/175 [==============================] - 13s 73ms/step - loss: 0.0957 - accuracy: 0.9723 - val_loss: 0.3459 - val_accuracy: 0.8743
Epoch 5/10
175/175 [==============================] - 15s 86ms/step - loss: 0.0883 - accuracy: 0.9745 - val_loss: 0.3966 - val_accuracy: 0.8734
Epoch 6/10
175/175 [==============================] - 14s 78ms/step - loss: 0.0841 - accuracy: 0.9764 - val_loss: 0.4113 - val_accuracy: 0.8661
Epoch 7/10
175/175 [==============================] - 12s 70ms/step - loss: 0.0897 - accuracy: 0.9771 - val_loss: 0.4312 - val_accurac

In [33]:
print(n)#
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

4
Epoch 1/10
219/219 [==============================] - 15s 68ms/step - loss: 0.2243 - accuracy: 0.9268 - val_loss: 0.3322 - val_accuracy: 0.8781
Epoch 2/10
219/219 [==============================] - 15s 67ms/step - loss: 0.1920 - accuracy: 0.9402 - val_loss: 0.3482 - val_accuracy: 0.8681
Epoch 3/10
219/219 [==============================] - 15s 68ms/step - loss: 0.1863 - accuracy: 0.9435 - val_loss: 0.3785 - val_accuracy: 0.8636
Epoch 4/10
219/219 [==============================] - 15s 68ms/step - loss: 0.1765 - accuracy: 0.9441 - val_loss: 0.3978 - val_accuracy: 0.8616
Epoch 5/10
219/219 [==============================] - 15s 67ms/step - loss: 0.1726 - accuracy: 0.9483 - val_loss: 0.4583 - val_accuracy: 0.8519
Epoch 6/10
219/219 [==============================] - 15s 68ms/step - loss: 0.1671 - accuracy: 0.9486 - val_loss: 0.4604 - val_accuracy: 0.8544
Epoch 7/10
219/219 [==============================] - 15s 68ms/step - loss: 0.1634 - accuracy: 0.9505 - val_loss: 0.4507 - val_accurac

In [34]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

5
Epoch 1/10
263/263 [==============================] - 18s 66ms/step - loss: 0.2247 - accuracy: 0.9334 - val_loss: 0.3302 - val_accuracy: 0.8769
Epoch 2/10
263/263 [==============================] - 17s 66ms/step - loss: 0.1985 - accuracy: 0.9422 - val_loss: 0.3460 - val_accuracy: 0.8708
Epoch 3/10
263/263 [==============================] - 21s 78ms/step - loss: 0.1969 - accuracy: 0.9432 - val_loss: 0.4124 - val_accuracy: 0.8499
Epoch 4/10
263/263 [==============================] - 18s 69ms/step - loss: 0.1907 - accuracy: 0.9465 - val_loss: 0.4751 - val_accuracy: 0.8370
Epoch 5/10
263/263 [==============================] - 22s 84ms/step - loss: 0.1928 - accuracy: 0.9473 - val_loss: 0.4664 - val_accuracy: 0.8361
Epoch 6/10
263/263 [==============================] - 19s 71ms/step - loss: 0.1930 - accuracy: 0.9461 - val_loss: 0.4846 - val_accuracy: 0.8231
Epoch 7/10
263/263 [==============================] - 18s 69ms/step - loss: 0.1909 - accuracy: 0.9480 - val_loss: 0.5185 - val_accurac

In [35]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

6
Epoch 1/10
307/307 [==============================] - 22s 69ms/step - loss: 0.2368 - accuracy: 0.9286 - val_loss: 0.3281 - val_accuracy: 0.8759
Epoch 2/10
307/307 [==============================] - 21s 69ms/step - loss: 0.2139 - accuracy: 0.9351 - val_loss: 0.3650 - val_accuracy: 0.8693
Epoch 3/10
307/307 [==============================] - 21s 70ms/step - loss: 0.2100 - accuracy: 0.9370 - val_loss: 0.3870 - val_accuracy: 0.8588
Epoch 4/10
307/307 [==============================] - 22s 71ms/step - loss: 0.2143 - accuracy: 0.9402 - val_loss: 0.4120 - val_accuracy: 0.8469
Epoch 5/10
307/307 [==============================] - 22s 71ms/step - loss: 0.2119 - accuracy: 0.9409 - val_loss: 0.4153 - val_accuracy: 0.8504
Epoch 6/10
307/307 [==============================] - 25s 80ms/step - loss: 0.2160 - accuracy: 0.9401 - val_loss: 0.5015 - val_accuracy: 0.8341
Epoch 7/10
307/307 [==============================] - 24s 79ms/step - loss: 0.2094 - accuracy: 0.9412 - val_loss: 0.4934 - val_accurac

In [36]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

7
Epoch 1/10
350/350 [==============================] - 26s 71ms/step - loss: 0.2276 - accuracy: 0.9329 - val_loss: 0.3442 - val_accuracy: 0.8709
Epoch 2/10
350/350 [==============================] - 24s 69ms/step - loss: 0.2066 - accuracy: 0.9399 - val_loss: 0.3697 - val_accuracy: 0.8600
Epoch 3/10
350/350 [==============================] - 24s 70ms/step - loss: 0.2020 - accuracy: 0.9408 - val_loss: 0.4195 - val_accuracy: 0.8504
Epoch 4/10
350/350 [==============================] - 24s 70ms/step - loss: 0.2086 - accuracy: 0.9406 - val_loss: 0.4276 - val_accuracy: 0.8493
Epoch 5/10
350/350 [==============================] - 25s 71ms/step - loss: 0.2121 - accuracy: 0.9404 - val_loss: 0.5097 - val_accuracy: 0.8189
Epoch 6/10
350/350 [==============================] - 26s 74ms/step - loss: 0.2160 - accuracy: 0.9413 - val_loss: 0.5070 - val_accuracy: 0.8131
Epoch 7/10
350/350 [==============================] - 25s 72ms/step - loss: 0.2173 - accuracy: 0.9423 - val_loss: 0.5934 - val_accurac

In [37]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

8
Epoch 1/10
394/394 [==============================] - 28s 69ms/step - loss: 0.2100 - accuracy: 0.9405 - val_loss: 0.3142 - val_accuracy: 0.8854
Epoch 2/10
394/394 [==============================] - 27s 68ms/step - loss: 0.1926 - accuracy: 0.9446 - val_loss: 0.4064 - val_accuracy: 0.8492
Epoch 3/10
394/394 [==============================] - 27s 69ms/step - loss: 0.1932 - accuracy: 0.9475 - val_loss: 0.4355 - val_accuracy: 0.8369
Epoch 4/10
394/394 [==============================] - 27s 69ms/step - loss: 0.2029 - accuracy: 0.9462 - val_loss: 0.4717 - val_accuracy: 0.8176
Epoch 5/10
394/394 [==============================] - 27s 69ms/step - loss: 0.1974 - accuracy: 0.9471 - val_loss: 0.9350 - val_accuracy: 0.7751
Epoch 6/10
394/394 [==============================] - 27s 69ms/step - loss: 0.2031 - accuracy: 0.9479 - val_loss: 0.5132 - val_accuracy: 0.8046
Epoch 7/10
394/394 [==============================] - 27s 69ms/step - loss: 0.2085 - accuracy: 0.9452 - val_loss: 0.6907 - val_accurac

In [38]:
print(n)#
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

9
Epoch 1/10
438/438 [==============================] - 30s 68ms/step - loss: 0.1974 - accuracy: 0.9442 - val_loss: 0.3562 - val_accuracy: 0.8716
Epoch 2/10
438/438 [==============================] - 30s 68ms/step - loss: 0.1796 - accuracy: 0.9490 - val_loss: 0.3674 - val_accuracy: 0.8696
Epoch 3/10
438/438 [==============================] - 30s 68ms/step - loss: 0.1873 - accuracy: 0.9482 - val_loss: 0.4848 - val_accuracy: 0.8239
Epoch 4/10
438/438 [==============================] - 30s 68ms/step - loss: 0.1904 - accuracy: 0.9486 - val_loss: 0.4888 - val_accuracy: 0.8302
Epoch 5/10
438/438 [==============================] - 30s 69ms/step - loss: 0.1960 - accuracy: 0.9496 - val_loss: 0.5685 - val_accuracy: 0.7891
Epoch 6/10
438/438 [==============================] - 30s 68ms/step - loss: 0.2014 - accuracy: 0.9486 - val_loss: 0.6200 - val_accuracy: 0.7802
Epoch 7/10
438/438 [==============================] - 30s 68ms/step - loss: 0.2023 - accuracy: 0.9494 - val_loss: 0.9686 - val_accurac

In [39]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

10
Epoch 1/10
482/482 [==============================] - 33s 68ms/step - loss: 0.2270 - accuracy: 0.9305 - val_loss: 0.4241 - val_accuracy: 0.8394
Epoch 2/10
482/482 [==============================] - 32s 67ms/step - loss: 0.2196 - accuracy: 0.9331 - val_loss: 0.3809 - val_accuracy: 0.8529
Epoch 3/10
482/482 [==============================] - 33s 67ms/step - loss: 0.2162 - accuracy: 0.9365 - val_loss: 0.4707 - val_accuracy: 0.8279
Epoch 4/10
482/482 [==============================] - 33s 69ms/step - loss: 0.2221 - accuracy: 0.9381 - val_loss: 0.6322 - val_accuracy: 0.7826
Epoch 5/10
482/482 [==============================] - 32s 67ms/step - loss: 0.2240 - accuracy: 0.9384 - val_loss: 0.6210 - val_accuracy: 0.7901
Epoch 6/10
482/482 [==============================] - 32s 67ms/step - loss: 0.2248 - accuracy: 0.9366 - val_loss: 0.6313 - val_accuracy: 0.7707
Epoch 7/10
482/482 [==============================] - 32s 67ms/step - loss: 0.2382 - accuracy: 0.9368 - val_loss: 0.6269 - val_accura

In [40]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

11
Epoch 1/10
525/525 [==============================] - 39s 73ms/step - loss: 0.2402 - accuracy: 0.9288 - val_loss: 0.3711 - val_accuracy: 0.8539
Epoch 2/10
525/525 [==============================] - 39s 73ms/step - loss: 0.2298 - accuracy: 0.9342 - val_loss: 0.4098 - val_accuracy: 0.8456
Epoch 3/10
525/525 [==============================] - 38s 73ms/step - loss: 0.2364 - accuracy: 0.9354 - val_loss: 0.4606 - val_accuracy: 0.8252
Epoch 4/10
525/525 [==============================] - 39s 74ms/step - loss: 0.2406 - accuracy: 0.9368 - val_loss: 0.4899 - val_accuracy: 0.8231
Epoch 5/10
525/525 [==============================] - 39s 73ms/step - loss: 0.2441 - accuracy: 0.9351 - val_loss: 0.4925 - val_accuracy: 0.8034
Epoch 6/10
525/525 [==============================] - 39s 74ms/step - loss: 0.2533 - accuracy: 0.9343 - val_loss: 0.6347 - val_accuracy: 0.7522
Epoch 7/10
525/525 [==============================] - 39s 73ms/step - loss: 0.2640 - accuracy: 0.9337 - val_loss: 0.7329 - val_accura

In [41]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

12
Epoch 1/10
569/569 [==============================] - 43s 74ms/step - loss: 0.2813 - accuracy: 0.9088 - val_loss: 0.3257 - val_accuracy: 0.8750
Epoch 2/10
569/569 [==============================] - 43s 75ms/step - loss: 0.2807 - accuracy: 0.9124 - val_loss: 0.3664 - val_accuracy: 0.8674
Epoch 3/10
569/569 [==============================] - 43s 75ms/step - loss: 0.2853 - accuracy: 0.9138 - val_loss: 0.3543 - val_accuracy: 0.8729
Epoch 4/10
569/569 [==============================] - 42s 74ms/step - loss: 0.2874 - accuracy: 0.9116 - val_loss: 0.4335 - val_accuracy: 0.8474
Epoch 5/10
569/569 [==============================] - 42s 74ms/step - loss: 0.2906 - accuracy: 0.9127 - val_loss: 0.6703 - val_accuracy: 0.7971
Epoch 6/10
569/569 [==============================] - 42s 74ms/step - loss: 0.3026 - accuracy: 0.9125 - val_loss: 0.5324 - val_accuracy: 0.7936
Epoch 7/10
569/569 [==============================] - 42s 74ms/step - loss: 0.3099 - accuracy: 0.9103 - val_loss: 0.7788 - val_accura

In [42]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

13
Epoch 1/10
613/613 [==============================] - 46s 74ms/step - loss: 0.3031 - accuracy: 0.9054 - val_loss: 0.3157 - val_accuracy: 0.8856
Epoch 2/10
613/613 [==============================] - 45s 73ms/step - loss: 0.2930 - accuracy: 0.9105 - val_loss: 0.3363 - val_accuracy: 0.8716
Epoch 3/10
613/613 [==============================] - 45s 73ms/step - loss: 0.2926 - accuracy: 0.9119 - val_loss: 0.3769 - val_accuracy: 0.8589
Epoch 4/10
613/613 [==============================] - 45s 73ms/step - loss: 0.2995 - accuracy: 0.9129 - val_loss: 0.4778 - val_accuracy: 0.8212
Epoch 5/10
613/613 [==============================] - 45s 74ms/step - loss: 0.3155 - accuracy: 0.9096 - val_loss: 0.5519 - val_accuracy: 0.8294
Epoch 6/10
613/613 [==============================] - 45s 74ms/step - loss: 0.3191 - accuracy: 0.9088 - val_loss: 0.5142 - val_accuracy: 0.7977
Epoch 7/10
613/613 [==============================] - 45s 73ms/step - loss: 0.3272 - accuracy: 0.9073 - val_loss: 0.6312 - val_accura

In [43]:
print(n)#
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

14
Epoch 1/10
657/657 [==============================] - 49s 73ms/step - loss: 0.3260 - accuracy: 0.8919 - val_loss: 0.2995 - val_accuracy: 0.8849
Epoch 2/10
657/657 [==============================] - 48s 73ms/step - loss: 0.3278 - accuracy: 0.8927 - val_loss: 0.3179 - val_accuracy: 0.8939
Epoch 3/10
657/657 [==============================] - 48s 73ms/step - loss: 0.3271 - accuracy: 0.8928 - val_loss: 0.3326 - val_accuracy: 0.8794
Epoch 4/10
657/657 [==============================] - 48s 73ms/step - loss: 0.3363 - accuracy: 0.8939 - val_loss: 0.3752 - val_accuracy: 0.8781
Epoch 5/10
657/657 [==============================] - 48s 74ms/step - loss: 0.3451 - accuracy: 0.8926 - val_loss: 0.3628 - val_accuracy: 0.8741
Epoch 6/10
657/657 [==============================] - 48s 73ms/step - loss: 0.3519 - accuracy: 0.8925 - val_loss: 0.4340 - val_accuracy: 0.8524
Epoch 7/10
657/657 [==============================] - 48s 73ms/step - loss: 0.3683 - accuracy: 0.8873 - val_loss: 0.4563 - val_accura

In [44]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

15
Epoch 1/10
700/700 [==============================] - 51s 72ms/step - loss: 0.3279 - accuracy: 0.8935 - val_loss: 0.2780 - val_accuracy: 0.9037
Epoch 2/10
700/700 [==============================] - 50s 72ms/step - loss: 0.3267 - accuracy: 0.8965 - val_loss: 0.2705 - val_accuracy: 0.9054
Epoch 3/10
700/700 [==============================] - 50s 72ms/step - loss: 0.3302 - accuracy: 0.8973 - val_loss: 0.3143 - val_accuracy: 0.8874
Epoch 4/10
700/700 [==============================] - 50s 72ms/step - loss: 0.3414 - accuracy: 0.8962 - val_loss: 0.3699 - val_accuracy: 0.8756
Epoch 5/10
700/700 [==============================] - 50s 72ms/step - loss: 0.3487 - accuracy: 0.8963 - val_loss: 0.3676 - val_accuracy: 0.8675
Epoch 6/10
700/700 [==============================] - 50s 72ms/step - loss: 0.3652 - accuracy: 0.8903 - val_loss: 0.3808 - val_accuracy: 0.8669
Epoch 7/10
700/700 [==============================] - 50s 72ms/step - loss: 0.3784 - accuracy: 0.8895 - val_loss: 0.4097 - val_accura

In [45]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

16
Epoch 1/10
744/744 [==============================] - 56s 74ms/step - loss: 0.3475 - accuracy: 0.8825 - val_loss: 0.2521 - val_accuracy: 0.9147
Epoch 2/10
744/744 [==============================] - 55s 74ms/step - loss: 0.3467 - accuracy: 0.8841 - val_loss: 0.2808 - val_accuracy: 0.8991
Epoch 3/10
744/744 [==============================] - 56s 76ms/step - loss: 0.3542 - accuracy: 0.8837 - val_loss: 0.2795 - val_accuracy: 0.9039
Epoch 4/10
744/744 [==============================] - 56s 75ms/step - loss: 0.3727 - accuracy: 0.8815 - val_loss: 0.3819 - val_accuracy: 0.8920
Epoch 5/10
744/744 [==============================] - 55s 74ms/step - loss: 0.3801 - accuracy: 0.8812 - val_loss: 0.3021 - val_accuracy: 0.9001
Epoch 6/10
744/744 [==============================] - 55s 74ms/step - loss: 0.3953 - accuracy: 0.8770 - val_loss: 0.3903 - val_accuracy: 0.8959
Epoch 7/10
744/744 [==============================] - 55s 74ms/step - loss: 0.3990 - accuracy: 0.8755 - val_loss: 0.3421 - val_accura

In [46]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

17
Epoch 1/10
788/788 [==============================] - 60s 76ms/step - loss: 0.3529 - accuracy: 0.8831 - val_loss: 0.2702 - val_accuracy: 0.9106
Epoch 2/10
788/788 [==============================] - 58s 74ms/step - loss: 0.3476 - accuracy: 0.8832 - val_loss: 0.2578 - val_accuracy: 0.9106
Epoch 3/10
788/788 [==============================] - 58s 73ms/step - loss: 0.3585 - accuracy: 0.8845 - val_loss: 0.2701 - val_accuracy: 0.9074
Epoch 4/10
788/788 [==============================] - 58s 74ms/step - loss: 0.3743 - accuracy: 0.8813 - val_loss: 0.3189 - val_accuracy: 0.8981
Epoch 5/10
788/788 [==============================] - 58s 74ms/step - loss: 0.3875 - accuracy: 0.8783 - val_loss: 0.2985 - val_accuracy: 0.9029
Epoch 6/10
788/788 [==============================] - 58s 74ms/step - loss: 0.3989 - accuracy: 0.8753 - val_loss: 0.3723 - val_accuracy: 0.8811
Epoch 7/10
788/788 [==============================] - 59s 74ms/step - loss: 0.4176 - accuracy: 0.8679 - val_loss: 0.3366 - val_accura

In [47]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

18
Epoch 1/10
832/832 [==============================] - 65s 77ms/step - loss: 0.3353 - accuracy: 0.8858 - val_loss: 0.2698 - val_accuracy: 0.9054
Epoch 2/10
832/832 [==============================] - 60s 72ms/step - loss: 0.3373 - accuracy: 0.8875 - val_loss: 0.2678 - val_accuracy: 0.9089
Epoch 3/10
832/832 [==============================] - 60s 72ms/step - loss: 0.3571 - accuracy: 0.8845 - val_loss: 0.2870 - val_accuracy: 0.9061
Epoch 4/10
832/832 [==============================] - 60s 72ms/step - loss: 0.3677 - accuracy: 0.8838 - val_loss: 0.3544 - val_accuracy: 0.9037
Epoch 5/10
832/832 [==============================] - 60s 72ms/step - loss: 0.3846 - accuracy: 0.8797 - val_loss: 0.3134 - val_accuracy: 0.8990
Epoch 6/10
832/832 [==============================] - 60s 72ms/step - loss: 0.3956 - accuracy: 0.8760 - val_loss: 0.2892 - val_accuracy: 0.9018
Epoch 7/10
832/832 [==============================] - 59s 71ms/step - loss: 0.4090 - accuracy: 0.8738 - val_loss: 0.3671 - val_accura

In [48]:
print(n)#
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

19
Epoch 1/10
875/875 [==============================] - 64s 72ms/step - loss: 0.3219 - accuracy: 0.8928 - val_loss: 0.2448 - val_accuracy: 0.9138
Epoch 2/10
875/875 [==============================] - 63s 72ms/step - loss: 0.3281 - accuracy: 0.8932 - val_loss: 0.2686 - val_accuracy: 0.9104
Epoch 3/10
875/875 [==============================] - 62s 71ms/step - loss: 0.3372 - accuracy: 0.8912 - val_loss: 0.2708 - val_accuracy: 0.9074
Epoch 4/10
875/875 [==============================] - 63s 72ms/step - loss: 0.3503 - accuracy: 0.8886 - val_loss: 0.2751 - val_accuracy: 0.9043
Epoch 5/10
875/875 [==============================] - 62s 71ms/step - loss: 0.3672 - accuracy: 0.8849 - val_loss: 0.3359 - val_accuracy: 0.8991
Epoch 6/10
875/875 [==============================] - 63s 72ms/step - loss: 0.3920 - accuracy: 0.8800 - val_loss: 0.2820 - val_accuracy: 0.9076
Epoch 7/10
875/875 [==============================] - 62s 71ms/step - loss: 0.3988 - accuracy: 0.8785 - val_loss: 0.3379 - val_accura

In [49]:
time.sleep(400)

In [ ]:
len(models_lsa)

In [50]:
#new_model_lsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_lsa_e2"
new_model_lsa_dir  = "D:/models/aug_22/"+dataset+"/C2/"+dataset+"_model_c2_may_lsa_e1"

i=0

for model in models_lsa:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_lsa_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_lsa_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_lsa_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_lsa_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_lsa_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_lsa_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_lsa_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_lsa_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written t

In [51]:
import gc

In [52]:
try:
    del lsa_values
    del top_images_by_lsa
    del top_labels_by_lsa
    del image_sets_lsa
    del label_sets_lsa
    del models_lsa
except:
    print("Error")

In [53]:
gc.collect()

328000

In [ ]:
loading = False

models_lsa = []

if loading:
    for i in range(10,20):
        #model_lsa_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_lsa_e1_"+str(i)
        print(model_lsa_dir)
        model =utils.My_model(dataset,True,model_lsa_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_lsa.append(model)
       
    

## Training guided by DSA

In [54]:
dsa_direction = "D:/guided-retraining/data/"+dataset+"/dsa_values.npy"

In [55]:
dsa_values = np.load(dsa_direction)

In [56]:
top_images_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),x_train_and_adversary)
top_labels_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),y_train_and_adversary)

In [57]:
m = n_data_points
n = 0
image_sets_dsa = []
label_sets_dsa = []


for i in range((len(top_images_by_dsa)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dsa)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dsa)%m))
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m])
    image_sets_dsa.append(top_images_by_dsa_n)
    label_sets_dsa.append(top_labels_by_dsa_n)
    print(len(top_images_by_dsa_n))
    n += m



0 :
0  ->  2800
2800
1 :
0  ->  5600
5600
2 :
0  ->  8400
8400
3 :
0  ->  11200
11200
4 :
0  ->  14000
14000
5 :
0  ->  16800
16800
6 :
0  ->  19600
19600
7 :
0  ->  22400
22400
8 :
0  ->  25200
25200
9 :
0  ->  28000
28000
10 :
0  ->  30800
30800
11 :
0  ->  33600
33600
12 :
0  ->  36400
36400
13 :
0  ->  39200
39200
14 :
0  ->  42000
42000
15 :
0  ->  44800
44800
16 :
0  ->  47600
47600
17 :
0  ->  50400
50400
18 :
0  ->  53200
53200
19 :
Last
0  ->  58798
55998


In [58]:
print(model_dir)

models_dsa = []
for i in range(len(label_sets_dsa)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_dsa.append(model)


D:/guided-retraining/models/model_fashion_2
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [59]:
n=0
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

0
Epoch 1/10
44/44 [==============================] - 7s 144ms/step - loss: 1.0105 - accuracy: 0.6218 - val_loss: 0.2991 - val_accuracy: 0.8884
Epoch 2/10
44/44 [==============================] - 6s 139ms/step - loss: 0.7661 - accuracy: 0.7132 - val_loss: 0.3013 - val_accuracy: 0.8834
Epoch 3/10
44/44 [==============================] - 6s 138ms/step - loss: 0.6739 - accuracy: 0.7425 - val_loss: 0.3019 - val_accuracy: 0.8843
Epoch 4/10
44/44 [==============================] - 6s 140ms/step - loss: 0.6420 - accuracy: 0.7604 - val_loss: 0.2942 - val_accuracy: 0.8861
Epoch 5/10
44/44 [==============================] - 6s 140ms/step - loss: 0.6009 - accuracy: 0.7682 - val_loss: 0.2893 - val_accuracy: 0.8891
Epoch 6/10
44/44 [==============================] - 6s 139ms/step - loss: 0.5507 - accuracy: 0.7968 - val_loss: 0.2956 - val_accuracy: 0.8912
Epoch 7/10
44/44 [==============================] - 6s 139ms/step - loss: 0.5163 - accuracy: 0.8125 - val_loss: 0.3003 - val_accuracy: 0.8900
Epoc

In [60]:
n=1
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

1
Epoch 1/10
88/88 [==============================] - 10s 102ms/step - loss: 0.6776 - accuracy: 0.7493 - val_loss: 0.2835 - val_accuracy: 0.8896
Epoch 2/10
88/88 [==============================] - 9s 100ms/step - loss: 0.5382 - accuracy: 0.8064 - val_loss: 0.2816 - val_accuracy: 0.8948
Epoch 3/10
88/88 [==============================] - 9s 102ms/step - loss: 0.5126 - accuracy: 0.8136 - val_loss: 0.2753 - val_accuracy: 0.8966
Epoch 4/10
88/88 [==============================] - 9s 102ms/step - loss: 0.4487 - accuracy: 0.8364 - val_loss: 0.2821 - val_accuracy: 0.8943
Epoch 5/10
88/88 [==============================] - 9s 100ms/step - loss: 0.4369 - accuracy: 0.8429 - val_loss: 0.2875 - val_accuracy: 0.8944
Epoch 6/10
88/88 [==============================] - 9s 100ms/step - loss: 0.4013 - accuracy: 0.8577 - val_loss: 0.2855 - val_accuracy: 0.8943
Epoch 7/10
88/88 [==============================] - 9s 101ms/step - loss: 0.3613 - accuracy: 0.8684 - val_loss: 0.2932 - val_accuracy: 0.8972
Epo

In [61]:
n=2
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

2
Epoch 1/10
132/132 [==============================] - 12s 87ms/step - loss: 0.5086 - accuracy: 0.8199 - val_loss: 0.2880 - val_accuracy: 0.8944
Epoch 2/10
132/132 [==============================] - 11s 86ms/step - loss: 0.4165 - accuracy: 0.8526 - val_loss: 0.2767 - val_accuracy: 0.8989
Epoch 3/10
132/132 [==============================] - 11s 86ms/step - loss: 0.3823 - accuracy: 0.8646 - val_loss: 0.2814 - val_accuracy: 0.8931
Epoch 4/10
132/132 [==============================] - 11s 83ms/step - loss: 0.3585 - accuracy: 0.8696 - val_loss: 0.2849 - val_accuracy: 0.9014
Epoch 5/10
132/132 [==============================] - 11s 86ms/step - loss: 0.3359 - accuracy: 0.8842 - val_loss: 0.2940 - val_accuracy: 0.8973
Epoch 6/10
132/132 [==============================] - 11s 86ms/step - loss: 0.3111 - accuracy: 0.8943 - val_loss: 0.3094 - val_accuracy: 0.8965
Epoch 7/10
132/132 [==============================] - 11s 86ms/step - loss: 0.2846 - accuracy: 0.9023 - val_loss: 0.2920 - val_accurac

In [62]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

3
Epoch 1/10
175/175 [==============================] - 15s 82ms/step - loss: 0.4513 - accuracy: 0.8403 - val_loss: 0.2778 - val_accuracy: 0.8970
Epoch 2/10
175/175 [==============================] - 14s 81ms/step - loss: 0.3805 - accuracy: 0.8662 - val_loss: 0.2754 - val_accuracy: 0.8988
Epoch 3/10
175/175 [==============================] - 14s 81ms/step - loss: 0.3551 - accuracy: 0.8750 - val_loss: 0.2659 - val_accuracy: 0.9035
Epoch 4/10
175/175 [==============================] - 14s 80ms/step - loss: 0.3433 - accuracy: 0.8799 - val_loss: 0.2805 - val_accuracy: 0.9026
Epoch 5/10
175/175 [==============================] - 14s 81ms/step - loss: 0.3334 - accuracy: 0.8887 - val_loss: 0.2769 - val_accuracy: 0.9089
Epoch 6/10
175/175 [==============================] - 14s 81ms/step - loss: 0.3038 - accuracy: 0.8949 - val_loss: 0.2883 - val_accuracy: 0.9061
Epoch 7/10
175/175 [==============================] - 14s 81ms/step - loss: 0.2882 - accuracy: 0.9024 - val_loss: 0.2820 - val_accurac

In [63]:
print(n)#
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

4
Epoch 1/10
219/219 [==============================] - 18s 78ms/step - loss: 0.4272 - accuracy: 0.8526 - val_loss: 0.2723 - val_accuracy: 0.9050
Epoch 2/10
219/219 [==============================] - 17s 78ms/step - loss: 0.3629 - accuracy: 0.8716 - val_loss: 0.2626 - val_accuracy: 0.9059
Epoch 3/10
219/219 [==============================] - 17s 78ms/step - loss: 0.3403 - accuracy: 0.8834 - val_loss: 0.2704 - val_accuracy: 0.9051
Epoch 4/10
219/219 [==============================] - 17s 79ms/step - loss: 0.3327 - accuracy: 0.8869 - val_loss: 0.2723 - val_accuracy: 0.9069
Epoch 5/10
219/219 [==============================] - 17s 79ms/step - loss: 0.3113 - accuracy: 0.8956 - val_loss: 0.2653 - val_accuracy: 0.9089
Epoch 6/10
219/219 [==============================] - 17s 79ms/step - loss: 0.3037 - accuracy: 0.9006 - val_loss: 0.2891 - val_accuracy: 0.8999
Epoch 7/10
219/219 [==============================] - 17s 79ms/step - loss: 0.2940 - accuracy: 0.9042 - val_loss: 0.2788 - val_accurac

In [64]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

5
Epoch 1/10
263/263 [==============================] - 21s 78ms/step - loss: 0.3945 - accuracy: 0.8655 - val_loss: 0.2809 - val_accuracy: 0.8993
Epoch 2/10
263/263 [==============================] - 20s 77ms/step - loss: 0.3583 - accuracy: 0.8752 - val_loss: 0.2657 - val_accuracy: 0.9079
Epoch 3/10
263/263 [==============================] - 20s 78ms/step - loss: 0.3353 - accuracy: 0.8833 - val_loss: 0.2669 - val_accuracy: 0.9094
Epoch 4/10
263/263 [==============================] - 21s 82ms/step - loss: 0.3318 - accuracy: 0.8888 - val_loss: 0.2759 - val_accuracy: 0.9065
Epoch 5/10
263/263 [==============================] - 22s 82ms/step - loss: 0.3056 - accuracy: 0.8953 - val_loss: 0.2751 - val_accuracy: 0.9054
Epoch 6/10
263/263 [==============================] - 21s 79ms/step - loss: 0.3054 - accuracy: 0.9002 - val_loss: 0.2935 - val_accuracy: 0.9094
Epoch 7/10
263/263 [==============================] - 21s 80ms/step - loss: 0.2890 - accuracy: 0.9040 - val_loss: 0.3107 - val_accurac

In [65]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

6
Epoch 1/10
307/307 [==============================] - 25s 79ms/step - loss: 0.3726 - accuracy: 0.8704 - val_loss: 0.2806 - val_accuracy: 0.9009
Epoch 2/10
307/307 [==============================] - 25s 81ms/step - loss: 0.3433 - accuracy: 0.8808 - val_loss: 0.2718 - val_accuracy: 0.9067
Epoch 3/10
307/307 [==============================] - 25s 81ms/step - loss: 0.3233 - accuracy: 0.8887 - val_loss: 0.2821 - val_accuracy: 0.9053
Epoch 4/10
307/307 [==============================] - 25s 82ms/step - loss: 0.3191 - accuracy: 0.8935 - val_loss: 0.2811 - val_accuracy: 0.9055
Epoch 5/10
307/307 [==============================] - 25s 81ms/step - loss: 0.3084 - accuracy: 0.8977 - val_loss: 0.3058 - val_accuracy: 0.9030
Epoch 6/10
307/307 [==============================] - 25s 81ms/step - loss: 0.3033 - accuracy: 0.8973 - val_loss: 0.2875 - val_accuracy: 0.9070
Epoch 7/10
307/307 [==============================] - 25s 81ms/step - loss: 0.3001 - accuracy: 0.9006 - val_loss: 0.2834 - val_accurac

In [66]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

7
Epoch 1/10
350/350 [==============================] - 28s 79ms/step - loss: 0.3650 - accuracy: 0.8752 - val_loss: 0.2611 - val_accuracy: 0.9062
Epoch 2/10
350/350 [==============================] - 28s 79ms/step - loss: 0.3276 - accuracy: 0.8876 - val_loss: 0.2579 - val_accuracy: 0.9088
Epoch 3/10
350/350 [==============================] - 28s 79ms/step - loss: 0.3291 - accuracy: 0.8892 - val_loss: 0.2793 - val_accuracy: 0.9054
Epoch 4/10
350/350 [==============================] - 27s 79ms/step - loss: 0.3200 - accuracy: 0.8933 - val_loss: 0.2624 - val_accuracy: 0.9092
Epoch 5/10
350/350 [==============================] - 28s 79ms/step - loss: 0.3080 - accuracy: 0.8977 - val_loss: 0.2799 - val_accuracy: 0.9062
Epoch 6/10
350/350 [==============================] - 28s 79ms/step - loss: 0.3067 - accuracy: 0.9006 - val_loss: 0.2785 - val_accuracy: 0.9061
Epoch 7/10
350/350 [==============================] - 28s 79ms/step - loss: 0.3063 - accuracy: 0.9017 - val_loss: 0.3070 - val_accurac

In [67]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

8
Epoch 1/10
394/394 [==============================] - 32s 78ms/step - loss: 0.3584 - accuracy: 0.8774 - val_loss: 0.2511 - val_accuracy: 0.9132
Epoch 2/10
394/394 [==============================] - 31s 78ms/step - loss: 0.3289 - accuracy: 0.8869 - val_loss: 0.2666 - val_accuracy: 0.9060
Epoch 3/10
394/394 [==============================] - 31s 78ms/step - loss: 0.3228 - accuracy: 0.8917 - val_loss: 0.2635 - val_accuracy: 0.9098
Epoch 4/10
394/394 [==============================] - 31s 79ms/step - loss: 0.3194 - accuracy: 0.8932 - val_loss: 0.2659 - val_accuracy: 0.9089
Epoch 5/10
394/394 [==============================] - 31s 80ms/step - loss: 0.3110 - accuracy: 0.8970 - val_loss: 0.2984 - val_accuracy: 0.9046
Epoch 6/10
394/394 [==============================] - 31s 79ms/step - loss: 0.3065 - accuracy: 0.8994 - val_loss: 0.2933 - val_accuracy: 0.9075
Epoch 7/10
394/394 [==============================] - 31s 78ms/step - loss: 0.3130 - accuracy: 0.8987 - val_loss: 0.3391 - val_accurac

In [68]:
print(n)#
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

9
Epoch 1/10
438/438 [==============================] - 35s 78ms/step - loss: 0.3414 - accuracy: 0.8815 - val_loss: 0.2723 - val_accuracy: 0.9093
Epoch 2/10
438/438 [==============================] - 34s 78ms/step - loss: 0.3243 - accuracy: 0.8884 - val_loss: 0.2711 - val_accuracy: 0.9085
Epoch 3/10
438/438 [==============================] - 34s 77ms/step - loss: 0.3188 - accuracy: 0.8914 - val_loss: 0.3104 - val_accuracy: 0.9025
Epoch 4/10
438/438 [==============================] - 34s 77ms/step - loss: 0.3130 - accuracy: 0.8964 - val_loss: 0.2673 - val_accuracy: 0.9105
Epoch 5/10
438/438 [==============================] - 34s 77ms/step - loss: 0.3225 - accuracy: 0.8950 - val_loss: 0.3043 - val_accuracy: 0.9110
Epoch 6/10
438/438 [==============================] - 34s 78ms/step - loss: 0.3159 - accuracy: 0.8977 - val_loss: 0.2743 - val_accuracy: 0.9096
Epoch 7/10
438/438 [==============================] - 34s 77ms/step - loss: 0.3285 - accuracy: 0.8979 - val_loss: 0.3310 - val_accurac

In [69]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

10
Epoch 1/10
482/482 [==============================] - 38s 77ms/step - loss: 0.3413 - accuracy: 0.8822 - val_loss: 0.2497 - val_accuracy: 0.9132
Epoch 2/10
482/482 [==============================] - 37s 78ms/step - loss: 0.3257 - accuracy: 0.8907 - val_loss: 0.2654 - val_accuracy: 0.9069
Epoch 3/10
482/482 [==============================] - 37s 77ms/step - loss: 0.3260 - accuracy: 0.8894 - val_loss: 0.2864 - val_accuracy: 0.9063
Epoch 4/10
482/482 [==============================] - 37s 77ms/step - loss: 0.3319 - accuracy: 0.8934 - val_loss: 0.2828 - val_accuracy: 0.9097
Epoch 5/10
482/482 [==============================] - 37s 77ms/step - loss: 0.3289 - accuracy: 0.8928 - val_loss: 0.2728 - val_accuracy: 0.9060
Epoch 6/10
482/482 [==============================] - 37s 77ms/step - loss: 0.3337 - accuracy: 0.8942 - val_loss: 0.3083 - val_accuracy: 0.8956
Epoch 7/10
482/482 [==============================] - 37s 77ms/step - loss: 0.3434 - accuracy: 0.8919 - val_loss: 0.3049 - val_accura

In [70]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

11
Epoch 1/10
525/525 [==============================] - 40s 75ms/step - loss: 0.3377 - accuracy: 0.8827 - val_loss: 0.2431 - val_accuracy: 0.9121
Epoch 2/10
525/525 [==============================] - 39s 75ms/step - loss: 0.3235 - accuracy: 0.8888 - val_loss: 0.2596 - val_accuracy: 0.9075
Epoch 3/10
525/525 [==============================] - 40s 76ms/step - loss: 0.3250 - accuracy: 0.8937 - val_loss: 0.2628 - val_accuracy: 0.9089
Epoch 4/10
525/525 [==============================] - 39s 75ms/step - loss: 0.3245 - accuracy: 0.8928 - val_loss: 0.2756 - val_accuracy: 0.9079
Epoch 5/10
525/525 [==============================] - 39s 75ms/step - loss: 0.3295 - accuracy: 0.8956 - val_loss: 0.3018 - val_accuracy: 0.9037
Epoch 6/10
525/525 [==============================] - 40s 76ms/step - loss: 0.3288 - accuracy: 0.8955 - val_loss: 0.2824 - val_accuracy: 0.9030
Epoch 7/10
525/525 [==============================] - 40s 76ms/step - loss: 0.3361 - accuracy: 0.8939 - val_loss: 0.3252 - val_accura

In [71]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

12
Epoch 1/10
569/569 [==============================] - 44s 76ms/step - loss: 0.3378 - accuracy: 0.8855 - val_loss: 0.2488 - val_accuracy: 0.9115
Epoch 2/10
569/569 [==============================] - 43s 76ms/step - loss: 0.3259 - accuracy: 0.8905 - val_loss: 0.2713 - val_accuracy: 0.9062
Epoch 3/10
569/569 [==============================] - 43s 76ms/step - loss: 0.3280 - accuracy: 0.8912 - val_loss: 0.2770 - val_accuracy: 0.9030
Epoch 4/10
569/569 [==============================] - 43s 76ms/step - loss: 0.3320 - accuracy: 0.8921 - val_loss: 0.2757 - val_accuracy: 0.9031
Epoch 5/10
569/569 [==============================] - 43s 76ms/step - loss: 0.3377 - accuracy: 0.8910 - val_loss: 0.2752 - val_accuracy: 0.9111
Epoch 6/10
569/569 [==============================] - 43s 76ms/step - loss: 0.3433 - accuracy: 0.8897 - val_loss: 0.2873 - val_accuracy: 0.9024
Epoch 7/10
569/569 [==============================] - 43s 76ms/step - loss: 0.3519 - accuracy: 0.8885 - val_loss: 0.3032 - val_accura

In [72]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

13
Epoch 1/10
613/613 [==============================] - 47s 76ms/step - loss: 0.3318 - accuracy: 0.8859 - val_loss: 0.2589 - val_accuracy: 0.9100
Epoch 2/10
613/613 [==============================] - 47s 76ms/step - loss: 0.3219 - accuracy: 0.8919 - val_loss: 0.2573 - val_accuracy: 0.9126
Epoch 3/10
613/613 [==============================] - 46s 76ms/step - loss: 0.3246 - accuracy: 0.8923 - val_loss: 0.2592 - val_accuracy: 0.9096
Epoch 4/10
613/613 [==============================] - 46s 75ms/step - loss: 0.3265 - accuracy: 0.8942 - val_loss: 0.2886 - val_accuracy: 0.9049
Epoch 5/10
613/613 [==============================] - 46s 76ms/step - loss: 0.3366 - accuracy: 0.8929 - val_loss: 0.2725 - val_accuracy: 0.9111
Epoch 6/10
613/613 [==============================] - 46s 75ms/step - loss: 0.3429 - accuracy: 0.8898 - val_loss: 0.3217 - val_accuracy: 0.8971
Epoch 7/10
613/613 [==============================] - 46s 76ms/step - loss: 0.3568 - accuracy: 0.8884 - val_loss: 0.3048 - val_accura

In [73]:
print(n)#
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

14
Epoch 1/10
657/657 [==============================] - 50s 75ms/step - loss: 0.3284 - accuracy: 0.8896 - val_loss: 0.2516 - val_accuracy: 0.9114
Epoch 2/10
657/657 [==============================] - 49s 75ms/step - loss: 0.3240 - accuracy: 0.8924 - val_loss: 0.2817 - val_accuracy: 0.9044
Epoch 3/10
657/657 [==============================] - 50s 76ms/step - loss: 0.3333 - accuracy: 0.8892 - val_loss: 0.2636 - val_accuracy: 0.9112
Epoch 4/10
657/657 [==============================] - 50s 76ms/step - loss: 0.3416 - accuracy: 0.8904 - val_loss: 0.2788 - val_accuracy: 0.9090
Epoch 5/10
657/657 [==============================] - 49s 75ms/step - loss: 0.3503 - accuracy: 0.8900 - val_loss: 0.5258 - val_accuracy: 0.8863
Epoch 6/10
657/657 [==============================] - 49s 75ms/step - loss: 0.3605 - accuracy: 0.8874 - val_loss: 0.2913 - val_accuracy: 0.9056
Epoch 7/10
657/657 [==============================] - 49s 75ms/step - loss: 0.3752 - accuracy: 0.8822 - val_loss: 0.3157 - val_accura

In [74]:
time.sleep(400)

In [75]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

15
Epoch 1/10
700/700 [==============================] - 48s 67ms/step - loss: 0.3249 - accuracy: 0.8891 - val_loss: 0.2547 - val_accuracy: 0.9119
Epoch 2/10
700/700 [==============================] - 47s 67ms/step - loss: 0.3238 - accuracy: 0.8898 - val_loss: 0.2596 - val_accuracy: 0.9124
Epoch 3/10
700/700 [==============================] - 47s 68ms/step - loss: 0.3302 - accuracy: 0.8922 - val_loss: 0.2808 - val_accuracy: 0.9061
Epoch 4/10
700/700 [==============================] - 47s 67ms/step - loss: 0.3483 - accuracy: 0.8891 - val_loss: 0.2774 - val_accuracy: 0.9104
Epoch 5/10
700/700 [==============================] - 47s 67ms/step - loss: 0.3644 - accuracy: 0.8862 - val_loss: 0.2984 - val_accuracy: 0.9044
Epoch 6/10
700/700 [==============================] - 47s 67ms/step - loss: 0.3731 - accuracy: 0.8852 - val_loss: 0.2802 - val_accuracy: 0.9051
Epoch 7/10
700/700 [==============================] - 47s 67ms/step - loss: 0.3814 - accuracy: 0.8832 - val_loss: 0.3056 - val_accura

In [76]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

16
Epoch 1/10
744/744 [==============================] - 52s 69ms/step - loss: 0.3203 - accuracy: 0.8912 - val_loss: 0.2659 - val_accuracy: 0.9050
Epoch 2/10
744/744 [==============================] - 51s 69ms/step - loss: 0.3261 - accuracy: 0.8911 - val_loss: 0.2774 - val_accuracy: 0.9107
Epoch 3/10
744/744 [==============================] - 52s 70ms/step - loss: 0.3343 - accuracy: 0.8914 - val_loss: 0.3297 - val_accuracy: 0.9109
Epoch 4/10
744/744 [==============================] - 53s 71ms/step - loss: 0.3477 - accuracy: 0.8890 - val_loss: 0.3340 - val_accuracy: 0.9020
Epoch 5/10
744/744 [==============================] - 53s 71ms/step - loss: 0.3574 - accuracy: 0.8864 - val_loss: 0.3078 - val_accuracy: 0.8969
Epoch 6/10
744/744 [==============================] - 54s 72ms/step - loss: 0.3698 - accuracy: 0.8854 - val_loss: 0.3304 - val_accuracy: 0.9019
Epoch 7/10
744/744 [==============================] - 55s 74ms/step - loss: 0.3902 - accuracy: 0.8812 - val_loss: 0.3037 - val_accura

In [77]:
time.sleep(400)

In [78]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

17
Epoch 1/10
788/788 [==============================] - 54s 68ms/step - loss: 0.3191 - accuracy: 0.8915 - val_loss: 0.2558 - val_accuracy: 0.9124
Epoch 2/10
788/788 [==============================] - 54s 68ms/step - loss: 0.3282 - accuracy: 0.8920 - val_loss: 0.2820 - val_accuracy: 0.9118
Epoch 3/10
788/788 [==============================] - 54s 68ms/step - loss: 0.3351 - accuracy: 0.8924 - val_loss: 0.2668 - val_accuracy: 0.9102
Epoch 4/10
788/788 [==============================] - 54s 68ms/step - loss: 0.3523 - accuracy: 0.8895 - val_loss: 0.3092 - val_accuracy: 0.8974
Epoch 5/10
788/788 [==============================] - 54s 68ms/step - loss: 0.3593 - accuracy: 0.8885 - val_loss: 0.3327 - val_accuracy: 0.8924
Epoch 6/10
788/788 [==============================] - 53s 68ms/step - loss: 0.3789 - accuracy: 0.8811 - val_loss: 0.2803 - val_accuracy: 0.9043
Epoch 7/10
788/788 [==============================] - 53s 68ms/step - loss: 0.3948 - accuracy: 0.8767 - val_loss: 0.3101 - val_accura

In [79]:
time.sleep(400)

In [80]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

18
Epoch 1/10
832/832 [==============================] - 57s 68ms/step - loss: 0.3207 - accuracy: 0.8915 - val_loss: 0.2563 - val_accuracy: 0.9116
Epoch 2/10
832/832 [==============================] - 56s 68ms/step - loss: 0.3268 - accuracy: 0.8923 - val_loss: 0.3291 - val_accuracy: 0.8963
Epoch 3/10
832/832 [==============================] - 56s 68ms/step - loss: 0.3397 - accuracy: 0.8900 - val_loss: 0.3438 - val_accuracy: 0.8892
Epoch 4/10
832/832 [==============================] - 56s 68ms/step - loss: 0.3548 - accuracy: 0.8888 - val_loss: 0.2724 - val_accuracy: 0.9075
Epoch 5/10
832/832 [==============================] - 57s 68ms/step - loss: 0.3717 - accuracy: 0.8851 - val_loss: 0.2941 - val_accuracy: 0.9035
Epoch 6/10
832/832 [==============================] - 56s 68ms/step - loss: 0.3816 - accuracy: 0.8825 - val_loss: 0.2819 - val_accuracy: 0.9025
Epoch 7/10
832/832 [==============================] - 56s 68ms/step - loss: 0.4005 - accuracy: 0.8776 - val_loss: 0.3040 - val_accura

In [81]:
time.sleep(400)

In [82]:
print(n)#
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

19
Epoch 1/10
875/875 [==============================] - 60s 68ms/step - loss: 0.3216 - accuracy: 0.8912 - val_loss: 0.2627 - val_accuracy: 0.9099
Epoch 2/10
875/875 [==============================] - 59s 68ms/step - loss: 0.3268 - accuracy: 0.8917 - val_loss: 0.2711 - val_accuracy: 0.9085
Epoch 3/10
875/875 [==============================] - 59s 68ms/step - loss: 0.3367 - accuracy: 0.8911 - val_loss: 0.2644 - val_accuracy: 0.9089
Epoch 4/10
875/875 [==============================] - 59s 67ms/step - loss: 0.3547 - accuracy: 0.8895 - val_loss: 0.2734 - val_accuracy: 0.9084
Epoch 5/10
875/875 [==============================] - 59s 67ms/step - loss: 0.3708 - accuracy: 0.8847 - val_loss: 0.3060 - val_accuracy: 0.8991
Epoch 6/10
875/875 [==============================] - 59s 67ms/step - loss: 0.3904 - accuracy: 0.8815 - val_loss: 0.2893 - val_accuracy: 0.9020
Epoch 7/10
875/875 [==============================] - 59s 67ms/step - loss: 0.4069 - accuracy: 0.8759 - val_loss: 0.3231 - val_accura

In [83]:
new_model_dsa_dir  = "D:/models/aug_22/"+dataset+"/C2/"+dataset+"_model_c2_may_dsa_e1"

i=0

for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_dsa_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_dsa_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_dsa_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_dsa_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_dsa_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_dsa_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_dsa_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_dsa_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written t

In [ ]:
loading = False

models_dsa = []

if loading:
    for i in range(20):
        model_dsa_dir = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_dsa_e2_"+str(i)
        print(model_dsa_dir)
        model =utils.My_model('gtsrb',True,model_dsa_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_dsa.append(model)
       
    

In [84]:
try:
    del dsa_values
    del top_images_by_dsa
    del top_labels_by_dsa
    del image_sets_dsa
    del label_sets_dsa
    del models_dsa
except:
    print("Error")

In [85]:
gc.collect()

327980

## Uncertainty

In [86]:
dg_direction = "D:/guided-retraining/data/"+dataset+"/deep_gini_values.npy"

deep_gini_values = np.load(dg_direction)


In [87]:
# Obtaining top n images by dg values
top_images_by_dg  = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),x_train_and_adversary)
top_labels_by_dg = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),y_train_and_adversary)

In [88]:
m = n_data_points
n = 0
image_sets_dg = []
label_sets_dg = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_dg)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dg)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dg)%m))
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m+(len(top_images_by_dg)%m)])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m+(len(top_images_by_dg)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m])
    image_sets_dg.append(top_images_by_dg_n)
    label_sets_dg.append(top_labels_by_dg_n)
    print(len(top_images_by_dg_n))
    n += m



0 :
0  ->  2800
2800
1 :
0  ->  5600
5600
2 :
0  ->  8400
8400
3 :
0  ->  11200
11200
4 :
0  ->  14000
14000
5 :
0  ->  16800
16800
6 :
0  ->  19600
19600
7 :
0  ->  22400
22400
8 :
0  ->  25200
25200
9 :
0  ->  28000
28000
10 :
0  ->  30800
30800
11 :
0  ->  33600
33600
12 :
0  ->  36400
36400
13 :
0  ->  39200
39200
14 :
0  ->  42000
42000
15 :
0  ->  44800
44800
16 :
0  ->  47600
47600
17 :
0  ->  50400
50400
18 :
0  ->  53200
53200
19 :
Last
0  ->  58798
55998


In [89]:
print(model_dir)
models_dg = []
#for i in range(len(label_sets_lsa)):
for i in range(len(label_sets_dg)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_dg.append(model)

D:/guided-retraining/models/model_fashion_2
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [90]:
n=0
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

0
Epoch 1/10
44/44 [==============================] - 7s 147ms/step - loss: 1.3012 - accuracy: 0.4596 - val_loss: 0.2494 - val_accuracy: 0.9149
Epoch 2/10
44/44 [==============================] - 6s 143ms/step - loss: 1.2084 - accuracy: 0.4871 - val_loss: 0.2624 - val_accuracy: 0.9121
Epoch 3/10
44/44 [==============================] - 6s 144ms/step - loss: 1.1430 - accuracy: 0.5243 - val_loss: 0.2633 - val_accuracy: 0.9148
Epoch 4/10
44/44 [==============================] - 6s 144ms/step - loss: 1.0973 - accuracy: 0.5536 - val_loss: 0.2621 - val_accuracy: 0.9115
Epoch 5/10
44/44 [==============================] - 6s 145ms/step - loss: 1.0576 - accuracy: 0.5693 - val_loss: 0.2717 - val_accuracy: 0.9061
Epoch 6/10
44/44 [==============================] - 6s 142ms/step - loss: 1.0490 - accuracy: 0.5543 - val_loss: 0.2719 - val_accuracy: 0.9091
Epoch 7/10
44/44 [==============================] - 6s 147ms/step - loss: 1.0448 - accuracy: 0.5754 - val_loss: 0.2714 - val_accuracy: 0.9145
Epoc

In [91]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

1
Epoch 1/10
88/88 [==============================] - 10s 109ms/step - loss: 1.1174 - accuracy: 0.5186 - val_loss: 0.2536 - val_accuracy: 0.9120
Epoch 2/10
88/88 [==============================] - 10s 110ms/step - loss: 1.0382 - accuracy: 0.5514 - val_loss: 0.2643 - val_accuracy: 0.9046
Epoch 3/10
88/88 [==============================] - 10s 109ms/step - loss: 0.9775 - accuracy: 0.5807 - val_loss: 0.2585 - val_accuracy: 0.9114
Epoch 4/10
88/88 [==============================] - 10s 110ms/step - loss: 0.9795 - accuracy: 0.5788 - val_loss: 0.2595 - val_accuracy: 0.9056
Epoch 5/10
88/88 [==============================] - 10s 111ms/step - loss: 0.9263 - accuracy: 0.6146 - val_loss: 0.2573 - val_accuracy: 0.9126
Epoch 6/10
88/88 [==============================] - 10s 110ms/step - loss: 0.9198 - accuracy: 0.6146 - val_loss: 0.2580 - val_accuracy: 0.9114
Epoch 7/10
88/88 [==============================] - 10s 111ms/step - loss: 0.8952 - accuracy: 0.6329 - val_loss: 0.2625 - val_accuracy: 0.90

In [92]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

2
Epoch 1/10
132/132 [==============================] - 14s 98ms/step - loss: 1.0239 - accuracy: 0.5669 - val_loss: 0.2497 - val_accuracy: 0.9176
Epoch 2/10
132/132 [==============================] - 13s 98ms/step - loss: 0.9514 - accuracy: 0.5931 - val_loss: 0.2535 - val_accuracy: 0.9082
Epoch 3/10
132/132 [==============================] - 13s 99ms/step - loss: 0.9127 - accuracy: 0.6106 - val_loss: 0.2609 - val_accuracy: 0.9130
Epoch 4/10
132/132 [==============================] - 13s 99ms/step - loss: 0.9096 - accuracy: 0.6143 - val_loss: 0.2515 - val_accuracy: 0.9093
Epoch 5/10
132/132 [==============================] - 13s 100ms/step - loss: 0.8678 - accuracy: 0.6336 - val_loss: 0.2553 - val_accuracy: 0.9117
Epoch 6/10
132/132 [==============================] - 13s 100ms/step - loss: 0.8386 - accuracy: 0.6431 - val_loss: 0.2645 - val_accuracy: 0.9101
Epoch 7/10
132/132 [==============================] - 13s 100ms/step - loss: 0.8378 - accuracy: 0.6445 - val_loss: 0.2485 - val_accu

In [93]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

3
Epoch 1/10
175/175 [==============================] - 17s 92ms/step - loss: 0.9462 - accuracy: 0.6091 - val_loss: 0.2417 - val_accuracy: 0.9126
Epoch 2/10
175/175 [==============================] - 16s 91ms/step - loss: 0.8892 - accuracy: 0.6271 - val_loss: 0.2409 - val_accuracy: 0.9200
Epoch 3/10
175/175 [==============================] - 16s 91ms/step - loss: 0.8594 - accuracy: 0.6472 - val_loss: 0.2570 - val_accuracy: 0.9119
Epoch 4/10
175/175 [==============================] - 16s 91ms/step - loss: 0.8402 - accuracy: 0.6571 - val_loss: 0.2564 - val_accuracy: 0.9093
Epoch 5/10
175/175 [==============================] - 16s 91ms/step - loss: 0.8004 - accuracy: 0.6708 - val_loss: 0.2475 - val_accuracy: 0.9144
Epoch 6/10
175/175 [==============================] - 16s 91ms/step - loss: 0.7754 - accuracy: 0.6829 - val_loss: 0.2465 - val_accuracy: 0.9174
Epoch 7/10
175/175 [==============================] - 16s 93ms/step - loss: 0.7773 - accuracy: 0.6902 - val_loss: 0.2449 - val_accurac

In [94]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

4
Epoch 1/10
219/219 [==============================] - 20s 88ms/step - loss: 0.8727 - accuracy: 0.6572 - val_loss: 0.2333 - val_accuracy: 0.9191
Epoch 2/10
219/219 [==============================] - 19s 88ms/step - loss: 0.8123 - accuracy: 0.6785 - val_loss: 0.2522 - val_accuracy: 0.9129
Epoch 3/10
219/219 [==============================] - 19s 88ms/step - loss: 0.7851 - accuracy: 0.6859 - val_loss: 0.2518 - val_accuracy: 0.9110
Epoch 4/10
219/219 [==============================] - 19s 89ms/step - loss: 0.7603 - accuracy: 0.7014 - val_loss: 0.2521 - val_accuracy: 0.9142
Epoch 5/10
219/219 [==============================] - 19s 88ms/step - loss: 0.7589 - accuracy: 0.7036 - val_loss: 0.2533 - val_accuracy: 0.9140
Epoch 6/10
219/219 [==============================] - 19s 88ms/step - loss: 0.7320 - accuracy: 0.7173 - val_loss: 0.2490 - val_accuracy: 0.9181
Epoch 7/10
219/219 [==============================] - 19s 88ms/step - loss: 0.7138 - accuracy: 0.7298 - val_loss: 0.2660 - val_accurac

In [95]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

5
Epoch 1/10
263/263 [==============================] - 23s 85ms/step - loss: 0.8022 - accuracy: 0.6944 - val_loss: 0.2415 - val_accuracy: 0.9159
Epoch 2/10
263/263 [==============================] - 22s 85ms/step - loss: 0.7525 - accuracy: 0.7113 - val_loss: 0.2432 - val_accuracy: 0.9186
Epoch 3/10
263/263 [==============================] - 22s 85ms/step - loss: 0.7317 - accuracy: 0.7210 - val_loss: 0.2543 - val_accuracy: 0.9121
Epoch 4/10
263/263 [==============================] - 23s 87ms/step - loss: 0.7133 - accuracy: 0.7352 - val_loss: 0.2413 - val_accuracy: 0.9180
Epoch 5/10
263/263 [==============================] - 23s 87ms/step - loss: 0.6945 - accuracy: 0.7395 - val_loss: 0.2485 - val_accuracy: 0.9166
Epoch 6/10
263/263 [==============================] - 22s 85ms/step - loss: 0.6956 - accuracy: 0.7435 - val_loss: 0.2501 - val_accuracy: 0.9186
Epoch 7/10
263/263 [==============================] - 22s 85ms/step - loss: 0.6659 - accuracy: 0.7577 - val_loss: 0.2869 - val_accurac

In [96]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

6
Epoch 1/10
307/307 [==============================] - 26s 83ms/step - loss: 0.7401 - accuracy: 0.7263 - val_loss: 0.2609 - val_accuracy: 0.9024
Epoch 2/10
307/307 [==============================] - 25s 83ms/step - loss: 0.6975 - accuracy: 0.7415 - val_loss: 0.2387 - val_accuracy: 0.9171
Epoch 3/10
307/307 [==============================] - 25s 83ms/step - loss: 0.6728 - accuracy: 0.7465 - val_loss: 0.2334 - val_accuracy: 0.9188
Epoch 4/10
307/307 [==============================] - 25s 83ms/step - loss: 0.6542 - accuracy: 0.7603 - val_loss: 0.2536 - val_accuracy: 0.9159
Epoch 5/10
307/307 [==============================] - 26s 83ms/step - loss: 0.6495 - accuracy: 0.7632 - val_loss: 0.2467 - val_accuracy: 0.9159
Epoch 6/10
307/307 [==============================] - 26s 83ms/step - loss: 0.6441 - accuracy: 0.7685 - val_loss: 0.2603 - val_accuracy: 0.9120
Epoch 7/10
307/307 [==============================] - 25s 83ms/step - loss: 0.6400 - accuracy: 0.7727 - val_loss: 0.2560 - val_accurac

In [97]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

7
Epoch 1/10
350/350 [==============================] - 29s 80ms/step - loss: 0.6800 - accuracy: 0.7557 - val_loss: 0.2345 - val_accuracy: 0.9150
Epoch 2/10
350/350 [==============================] - 28s 80ms/step - loss: 0.6435 - accuracy: 0.7671 - val_loss: 0.2449 - val_accuracy: 0.9150
Epoch 3/10
350/350 [==============================] - 28s 80ms/step - loss: 0.6286 - accuracy: 0.7741 - val_loss: 0.2386 - val_accuracy: 0.9164
Epoch 4/10
350/350 [==============================] - 28s 80ms/step - loss: 0.6161 - accuracy: 0.7798 - val_loss: 0.2472 - val_accuracy: 0.9165
Epoch 5/10
350/350 [==============================] - 28s 80ms/step - loss: 0.6100 - accuracy: 0.7869 - val_loss: 0.2530 - val_accuracy: 0.9148
Epoch 6/10
350/350 [==============================] - 28s 80ms/step - loss: 0.6050 - accuracy: 0.7887 - val_loss: 0.2551 - val_accuracy: 0.9144
Epoch 7/10
350/350 [==============================] - 28s 81ms/step - loss: 0.6014 - accuracy: 0.7918 - val_loss: 0.2606 - val_accurac

In [98]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

8
Epoch 1/10
394/394 [==============================] - 32s 80ms/step - loss: 0.6357 - accuracy: 0.7756 - val_loss: 0.2419 - val_accuracy: 0.9133
Epoch 2/10
394/394 [==============================] - 31s 80ms/step - loss: 0.5979 - accuracy: 0.7866 - val_loss: 0.2439 - val_accuracy: 0.9143
Epoch 3/10
394/394 [==============================] - 32s 80ms/step - loss: 0.5802 - accuracy: 0.7960 - val_loss: 0.2403 - val_accuracy: 0.9168
Epoch 4/10
394/394 [==============================] - 32s 80ms/step - loss: 0.5748 - accuracy: 0.8004 - val_loss: 0.2558 - val_accuracy: 0.9155
Epoch 5/10
394/394 [==============================] - 32s 80ms/step - loss: 0.5687 - accuracy: 0.8017 - val_loss: 0.2395 - val_accuracy: 0.9179
Epoch 6/10
394/394 [==============================] - 32s 81ms/step - loss: 0.5630 - accuracy: 0.8084 - val_loss: 0.2590 - val_accuracy: 0.9134
Epoch 7/10
394/394 [==============================] - 32s 80ms/step - loss: 0.5588 - accuracy: 0.8112 - val_loss: 0.2532 - val_accurac

In [99]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

9
Epoch 1/10
438/438 [==============================] - 36s 79ms/step - loss: 0.5779 - accuracy: 0.7984 - val_loss: 0.2433 - val_accuracy: 0.9141
Epoch 2/10
438/438 [==============================] - 35s 79ms/step - loss: 0.5606 - accuracy: 0.8036 - val_loss: 0.2564 - val_accuracy: 0.9079
Epoch 3/10
438/438 [==============================] - 35s 79ms/step - loss: 0.5410 - accuracy: 0.8137 - val_loss: 0.2471 - val_accuracy: 0.9153
Epoch 4/10
438/438 [==============================] - 35s 79ms/step - loss: 0.5403 - accuracy: 0.8177 - val_loss: 0.2597 - val_accuracy: 0.9121
Epoch 5/10
438/438 [==============================] - 35s 79ms/step - loss: 0.5382 - accuracy: 0.8178 - val_loss: 0.2497 - val_accuracy: 0.9157
Epoch 6/10
438/438 [==============================] - 35s 79ms/step - loss: 0.5408 - accuracy: 0.8189 - val_loss: 0.2497 - val_accuracy: 0.9141
Epoch 7/10
438/438 [==============================] - 35s 79ms/step - loss: 0.5440 - accuracy: 0.8179 - val_loss: 0.2742 - val_accurac

In [100]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

10
Epoch 1/10
482/482 [==============================] - 39s 78ms/step - loss: 0.5364 - accuracy: 0.8111 - val_loss: 0.2567 - val_accuracy: 0.9071
Epoch 2/10
482/482 [==============================] - 38s 78ms/step - loss: 0.5209 - accuracy: 0.8199 - val_loss: 0.2486 - val_accuracy: 0.9161
Epoch 3/10
482/482 [==============================] - 37s 78ms/step - loss: 0.5152 - accuracy: 0.8219 - val_loss: 0.2621 - val_accuracy: 0.9092
Epoch 4/10
482/482 [==============================] - 38s 79ms/step - loss: 0.5074 - accuracy: 0.8302 - val_loss: 0.2565 - val_accuracy: 0.9131
Epoch 5/10
482/482 [==============================] - 38s 78ms/step - loss: 0.5147 - accuracy: 0.8298 - val_loss: 0.2937 - val_accuracy: 0.9063
Epoch 6/10
482/482 [==============================] - 38s 78ms/step - loss: 0.5145 - accuracy: 0.8314 - val_loss: 0.2535 - val_accuracy: 0.9137
Epoch 7/10
482/482 [==============================] - 38s 78ms/step - loss: 0.5186 - accuracy: 0.8301 - val_loss: 0.2942 - val_accura

In [101]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

11
Epoch 1/10
525/525 [==============================] - 41s 77ms/step - loss: 0.5073 - accuracy: 0.8276 - val_loss: 0.2388 - val_accuracy: 0.9157
Epoch 2/10
525/525 [==============================] - 40s 76ms/step - loss: 0.4793 - accuracy: 0.8367 - val_loss: 0.2528 - val_accuracy: 0.9131
Epoch 3/10
525/525 [==============================] - 41s 77ms/step - loss: 0.4862 - accuracy: 0.8365 - val_loss: 0.2611 - val_accuracy: 0.9081
Epoch 4/10
525/525 [==============================] - 40s 76ms/step - loss: 0.4763 - accuracy: 0.8393 - val_loss: 0.2597 - val_accuracy: 0.9131
Epoch 5/10
525/525 [==============================] - 40s 76ms/step - loss: 0.4860 - accuracy: 0.8411 - val_loss: 0.2533 - val_accuracy: 0.9157
Epoch 6/10
525/525 [==============================] - 40s 76ms/step - loss: 0.4939 - accuracy: 0.8398 - val_loss: 0.2689 - val_accuracy: 0.9121
Epoch 7/10
525/525 [==============================] - 40s 76ms/step - loss: 0.5004 - accuracy: 0.8390 - val_loss: 0.2734 - val_accura

In [102]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

12
Epoch 1/10
569/569 [==============================] - 45s 77ms/step - loss: 0.4723 - accuracy: 0.8381 - val_loss: 0.2377 - val_accuracy: 0.9173
Epoch 2/10
569/569 [==============================] - 44s 77ms/step - loss: 0.4612 - accuracy: 0.8442 - val_loss: 0.2590 - val_accuracy: 0.9129
Epoch 3/10
569/569 [==============================] - 46s 80ms/step - loss: 0.4549 - accuracy: 0.8471 - val_loss: 0.2503 - val_accuracy: 0.9195
Epoch 4/10
569/569 [==============================] - 44s 77ms/step - loss: 0.4616 - accuracy: 0.8500 - val_loss: 0.2417 - val_accuracy: 0.9197
Epoch 5/10
569/569 [==============================] - 44s 78ms/step - loss: 0.4706 - accuracy: 0.8489 - val_loss: 0.2546 - val_accuracy: 0.9160
Epoch 6/10
569/569 [==============================] - 44s 77ms/step - loss: 0.4736 - accuracy: 0.8490 - val_loss: 0.2587 - val_accuracy: 0.9110
Epoch 7/10
569/569 [==============================] - 44s 77ms/step - loss: 0.4817 - accuracy: 0.8491 - val_loss: 0.3117 - val_accura

In [103]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

13
Epoch 1/10
613/613 [==============================] - 49s 78ms/step - loss: 0.4403 - accuracy: 0.8489 - val_loss: 0.2415 - val_accuracy: 0.9129
Epoch 2/10
613/613 [==============================] - 47s 76ms/step - loss: 0.4267 - accuracy: 0.8558 - val_loss: 0.2493 - val_accuracy: 0.9125
Epoch 3/10
613/613 [==============================] - 46s 75ms/step - loss: 0.4370 - accuracy: 0.8555 - val_loss: 0.2563 - val_accuracy: 0.9109
Epoch 4/10
613/613 [==============================] - 47s 77ms/step - loss: 0.4453 - accuracy: 0.8561 - val_loss: 0.2713 - val_accuracy: 0.9056
Epoch 5/10
613/613 [==============================] - 47s 77ms/step - loss: 0.4500 - accuracy: 0.8567 - val_loss: 0.2548 - val_accuracy: 0.9115
Epoch 6/10
613/613 [==============================] - 47s 76ms/step - loss: 0.4620 - accuracy: 0.8524 - val_loss: 0.2830 - val_accuracy: 0.9087
Epoch 7/10
613/613 [==============================] - 47s 77ms/step - loss: 0.4729 - accuracy: 0.8491 - val_loss: 0.3253 - val_accura

In [104]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

14
Epoch 1/10
657/657 [==============================] - 51s 76ms/step - loss: 0.4176 - accuracy: 0.8597 - val_loss: 0.2417 - val_accuracy: 0.9149
Epoch 2/10
657/657 [==============================] - 50s 76ms/step - loss: 0.4071 - accuracy: 0.8644 - val_loss: 0.2442 - val_accuracy: 0.9154
Epoch 3/10
657/657 [==============================] - 50s 76ms/step - loss: 0.4126 - accuracy: 0.8632 - val_loss: 0.2553 - val_accuracy: 0.9146
Epoch 4/10
657/657 [==============================] - 50s 76ms/step - loss: 0.4239 - accuracy: 0.8644 - val_loss: 0.2699 - val_accuracy: 0.9110
Epoch 5/10
657/657 [==============================] - 50s 76ms/step - loss: 0.4310 - accuracy: 0.8630 - val_loss: 0.2788 - val_accuracy: 0.9131
Epoch 6/10
657/657 [==============================] - 51s 78ms/step - loss: 0.4353 - accuracy: 0.8613 - val_loss: 0.2924 - val_accuracy: 0.9104
Epoch 7/10
657/657 [==============================] - 49s 75ms/step - loss: 0.4512 - accuracy: 0.8589 - val_loss: 0.2718 - val_accura

In [105]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

15
Epoch 1/10
700/700 [==============================] - 52s 74ms/step - loss: 0.3871 - accuracy: 0.8676 - val_loss: 0.2483 - val_accuracy: 0.9134
Epoch 2/10
700/700 [==============================] - 52s 75ms/step - loss: 0.3890 - accuracy: 0.8701 - val_loss: 0.2523 - val_accuracy: 0.9136
Epoch 3/10
700/700 [==============================] - 52s 74ms/step - loss: 0.3917 - accuracy: 0.8717 - val_loss: 0.2559 - val_accuracy: 0.9137
Epoch 4/10
700/700 [==============================] - 52s 74ms/step - loss: 0.4055 - accuracy: 0.8685 - val_loss: 0.2602 - val_accuracy: 0.9143
Epoch 5/10
700/700 [==============================] - 52s 74ms/step - loss: 0.4146 - accuracy: 0.8680 - val_loss: 0.2792 - val_accuracy: 0.9093
Epoch 6/10
700/700 [==============================] - 52s 74ms/step - loss: 0.4290 - accuracy: 0.8643 - val_loss: 0.2854 - val_accuracy: 0.9044
Epoch 7/10
700/700 [==============================] - 52s 74ms/step - loss: 0.4467 - accuracy: 0.8605 - val_loss: 0.2734 - val_accura

In [106]:
time.sleep(400)

In [107]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

16
Epoch 1/10
744/744 [==============================] - 52s 69ms/step - loss: 0.3686 - accuracy: 0.8741 - val_loss: 0.2472 - val_accuracy: 0.9126
Epoch 2/10
744/744 [==============================] - 51s 69ms/step - loss: 0.3693 - accuracy: 0.8764 - val_loss: 0.2725 - val_accuracy: 0.9088
Epoch 3/10
744/744 [==============================] - 52s 69ms/step - loss: 0.3765 - accuracy: 0.8757 - val_loss: 0.3058 - val_accuracy: 0.9028
Epoch 4/10
744/744 [==============================] - 51s 69ms/step - loss: 0.3931 - accuracy: 0.8748 - val_loss: 0.2857 - val_accuracy: 0.9076
Epoch 5/10
744/744 [==============================] - 51s 69ms/step - loss: 0.4049 - accuracy: 0.8706 - val_loss: 0.2907 - val_accuracy: 0.9064
Epoch 6/10
744/744 [==============================] - 51s 69ms/step - loss: 0.4182 - accuracy: 0.8705 - val_loss: 0.3190 - val_accuracy: 0.9024
Epoch 7/10
744/744 [==============================] - 51s 69ms/step - loss: 0.4359 - accuracy: 0.8649 - val_loss: 0.3072 - val_accura

In [108]:
time.sleep(400)

In [109]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

17
Epoch 1/10
788/788 [==============================] - 54s 68ms/step - loss: 0.3539 - accuracy: 0.8808 - val_loss: 0.2456 - val_accuracy: 0.9143
Epoch 2/10
788/788 [==============================] - 54s 69ms/step - loss: 0.3572 - accuracy: 0.8832 - val_loss: 0.2597 - val_accuracy: 0.9126
Epoch 3/10
788/788 [==============================] - 54s 69ms/step - loss: 0.3684 - accuracy: 0.8796 - val_loss: 0.2974 - val_accuracy: 0.9092
Epoch 4/10
788/788 [==============================] - 54s 68ms/step - loss: 0.3795 - accuracy: 0.8778 - val_loss: 0.2886 - val_accuracy: 0.8969
Epoch 5/10
788/788 [==============================] - 54s 69ms/step - loss: 0.3934 - accuracy: 0.8772 - val_loss: 0.2777 - val_accuracy: 0.9038
Epoch 6/10
788/788 [==============================] - 54s 69ms/step - loss: 0.4022 - accuracy: 0.8747 - val_loss: 0.3410 - val_accuracy: 0.8926
Epoch 7/10
788/788 [==============================] - 54s 68ms/step - loss: 0.4246 - accuracy: 0.8709 - val_loss: 0.3028 - val_accura

In [110]:
time.sleep(400)

In [111]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

18
Epoch 1/10
832/832 [==============================] - 58s 68ms/step - loss: 0.3336 - accuracy: 0.8864 - val_loss: 0.2622 - val_accuracy: 0.9120
Epoch 2/10
832/832 [==============================] - 57s 68ms/step - loss: 0.3381 - accuracy: 0.8879 - val_loss: 0.2801 - val_accuracy: 0.9089
Epoch 3/10
832/832 [==============================] - 57s 68ms/step - loss: 0.3575 - accuracy: 0.8844 - val_loss: 0.3399 - val_accuracy: 0.9093
Epoch 4/10
832/832 [==============================] - 57s 68ms/step - loss: 0.3714 - accuracy: 0.8835 - val_loss: 0.2752 - val_accuracy: 0.9090
Epoch 5/10
832/832 [==============================] - 57s 68ms/step - loss: 0.3915 - accuracy: 0.8785 - val_loss: 0.3122 - val_accuracy: 0.8979
Epoch 6/10
832/832 [==============================] - 56s 68ms/step - loss: 0.4066 - accuracy: 0.8763 - val_loss: 0.2836 - val_accuracy: 0.9031
Epoch 7/10
832/832 [==============================] - 57s 68ms/step - loss: 0.4122 - accuracy: 0.8713 - val_loss: 0.4843 - val_accura

In [112]:
time.sleep(400)

In [113]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

19
Epoch 1/10
875/875 [==============================] - 61s 68ms/step - loss: 0.3219 - accuracy: 0.8925 - val_loss: 0.2581 - val_accuracy: 0.9139
Epoch 2/10
875/875 [==============================] - 60s 68ms/step - loss: 0.3245 - accuracy: 0.8931 - val_loss: 0.2679 - val_accuracy: 0.9070
Epoch 3/10
875/875 [==============================] - 60s 68ms/step - loss: 0.3416 - accuracy: 0.8908 - val_loss: 0.3042 - val_accuracy: 0.9030
Epoch 4/10
875/875 [==============================] - 60s 68ms/step - loss: 0.3595 - accuracy: 0.8861 - val_loss: 0.2842 - val_accuracy: 0.9087
Epoch 5/10
875/875 [==============================] - 60s 68ms/step - loss: 0.3750 - accuracy: 0.8826 - val_loss: 0.3014 - val_accuracy: 0.8950
Epoch 6/10
875/875 [==============================] - 60s 68ms/step - loss: 0.3830 - accuracy: 0.8793 - val_loss: 0.2919 - val_accuracy: 0.9012
Epoch 7/10
875/875 [==============================] - 60s 68ms/step - loss: 0.4031 - accuracy: 0.8760 - val_loss: 0.3532 - val_accura

In [114]:
new_model_dg_dir  = "D:/models/aug_22/"+dataset+"/C2/"+dataset+"_model_c2_may_dg_e1"

i=0

for model in models_dg:
    model.save(new_model_dg_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_dg_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_dg_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_dg_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_dg_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_dg_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_dg_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_dg_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_dg_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/mo

In [ ]:
loading = False

models_dg = []

if loading:
    for i in range(5):
        model_dg_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_gn_e1_"+str(i)
        print(model_dg_dir)
        model =utils.My_model('gtsrb',True,model_dg_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_dg.append(model)
       
    

In [115]:
try:
    del deep_gini_values
    del top_images_by_dg
    del top_labels_by_dg
    del image_sets_dg
    del label_sets_dg
    del models_dg
except:
    print("Error")

In [116]:
import gc
gc.collect()

382925

### Softmax 

In [117]:
#softmax values
se_direction = "D:/guided-retraining/data/"+dataset+"/softmax_values.npy"
se_values = np.load(se_direction)


In [118]:
# Obtaining top n images by LSA values
top_images_by_se  = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),x_train_and_adversary)
top_labels_by_se = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),y_train_and_adversary)

In [119]:
m = n_data_points

n = 0
image_sets_se = []
label_sets_se = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_se)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_se)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_se)%m))
        top_images_by_se_n = np.array(top_images_by_se[:n+m+(len(top_images_by_se)%m)])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m+(len(top_images_by_se)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_se_n = np.array(top_images_by_se[:n+m])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m])
    image_sets_se.append(top_images_by_se_n)
    label_sets_se.append(top_labels_by_se_n)
    print(len(top_images_by_se_n))
    n += m



0 :
0  ->  2800
2800
1 :
0  ->  5600
5600
2 :
0  ->  8400
8400
3 :
0  ->  11200
11200
4 :
0  ->  14000
14000
5 :
0  ->  16800
16800
6 :
0  ->  19600
19600
7 :
0  ->  22400
22400
8 :
0  ->  25200
25200
9 :
0  ->  28000
28000
10 :
0  ->  30800
30800
11 :
0  ->  33600
33600
12 :
0  ->  36400
36400
13 :
0  ->  39200
39200
14 :
0  ->  42000
42000
15 :
0  ->  44800
44800
16 :
0  ->  47600
47600
17 :
0  ->  50400
50400
18 :
0  ->  53200
53200
19 :
Last
0  ->  58798
55998


In [120]:
print(model_dir)
models_se = []
for i in range(len(label_sets_se)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_se.append(model)

D:/guided-retraining/models/model_fashion_2
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [121]:
n=0
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

0
Epoch 1/10
44/44 [==============================] - 7s 150ms/step - loss: 1.3550 - accuracy: 0.4568 - val_loss: 0.2580 - val_accuracy: 0.9104
Epoch 2/10
44/44 [==============================] - 6s 146ms/step - loss: 1.1813 - accuracy: 0.5161 - val_loss: 0.2591 - val_accuracy: 0.9105
Epoch 3/10
44/44 [==============================] - 6s 147ms/step - loss: 1.1379 - accuracy: 0.5314 - val_loss: 0.2599 - val_accuracy: 0.9121
Epoch 4/10
44/44 [==============================] - 6s 147ms/step - loss: 1.0842 - accuracy: 0.5654 - val_loss: 0.2654 - val_accuracy: 0.9111
Epoch 5/10
44/44 [==============================] - 6s 147ms/step - loss: 1.0722 - accuracy: 0.5754 - val_loss: 0.2839 - val_accuracy: 0.9074
Epoch 6/10
44/44 [==============================] - 6s 147ms/step - loss: 1.0329 - accuracy: 0.5864 - val_loss: 0.2775 - val_accuracy: 0.9037
Epoch 7/10
44/44 [==============================] - 7s 154ms/step - loss: 0.9824 - accuracy: 0.6068 - val_loss: 0.2730 - val_accuracy: 0.9081
Epoc

In [122]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

1
Epoch 1/10
88/88 [==============================] - 11s 110ms/step - loss: 1.1470 - accuracy: 0.5209 - val_loss: 0.2504 - val_accuracy: 0.9154
Epoch 2/10
88/88 [==============================] - 10s 111ms/step - loss: 1.0480 - accuracy: 0.5666 - val_loss: 0.2632 - val_accuracy: 0.9076
Epoch 3/10
88/88 [==============================] - 10s 110ms/step - loss: 1.0239 - accuracy: 0.5811 - val_loss: 0.2555 - val_accuracy: 0.9126
Epoch 4/10
88/88 [==============================] - 10s 111ms/step - loss: 0.9799 - accuracy: 0.5979 - val_loss: 0.2609 - val_accuracy: 0.9110
Epoch 5/10
88/88 [==============================] - 10s 112ms/step - loss: 0.9602 - accuracy: 0.6016 - val_loss: 0.2615 - val_accuracy: 0.9069
Epoch 6/10
88/88 [==============================] - 10s 112ms/step - loss: 0.9360 - accuracy: 0.6204 - val_loss: 0.2625 - val_accuracy: 0.9133
Epoch 7/10
88/88 [==============================] - 10s 112ms/step - loss: 0.9149 - accuracy: 0.6343 - val_loss: 0.2824 - val_accuracy: 0.90

In [123]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

2
Epoch 1/10
132/132 [==============================] - 14s 99ms/step - loss: 1.0507 - accuracy: 0.5564 - val_loss: 0.2511 - val_accuracy: 0.9114
Epoch 2/10
132/132 [==============================] - 13s 99ms/step - loss: 0.9693 - accuracy: 0.5946 - val_loss: 0.2685 - val_accuracy: 0.9056
Epoch 3/10
132/132 [==============================] - 13s 100ms/step - loss: 0.9310 - accuracy: 0.6188 - val_loss: 0.2597 - val_accuracy: 0.9046
Epoch 4/10
132/132 [==============================] - 13s 101ms/step - loss: 0.8971 - accuracy: 0.6340 - val_loss: 0.2607 - val_accuracy: 0.9034
Epoch 5/10
132/132 [==============================] - 13s 101ms/step - loss: 0.8786 - accuracy: 0.6399 - val_loss: 0.2459 - val_accuracy: 0.9141
Epoch 6/10
132/132 [==============================] - 13s 101ms/step - loss: 0.8517 - accuracy: 0.6498 - val_loss: 0.2610 - val_accuracy: 0.9046
Epoch 7/10
132/132 [==============================] - 13s 101ms/step - loss: 0.8345 - accuracy: 0.6592 - val_loss: 0.2594 - val_ac

In [124]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

3
Epoch 1/10
175/175 [==============================] - 17s 93ms/step - loss: 0.9577 - accuracy: 0.6013 - val_loss: 0.2484 - val_accuracy: 0.9095
Epoch 2/10
175/175 [==============================] - 17s 95ms/step - loss: 0.8852 - accuracy: 0.6322 - val_loss: 0.2420 - val_accuracy: 0.9178
Epoch 3/10
175/175 [==============================] - 16s 93ms/step - loss: 0.8492 - accuracy: 0.6537 - val_loss: 0.2413 - val_accuracy: 0.9191
Epoch 4/10
175/175 [==============================] - 16s 93ms/step - loss: 0.8369 - accuracy: 0.6597 - val_loss: 0.2442 - val_accuracy: 0.9155
Epoch 5/10
175/175 [==============================] - 16s 93ms/step - loss: 0.8165 - accuracy: 0.6729 - val_loss: 0.2485 - val_accuracy: 0.9121
Epoch 6/10
175/175 [==============================] - 16s 93ms/step - loss: 0.7885 - accuracy: 0.6863 - val_loss: 0.2509 - val_accuracy: 0.9138
Epoch 7/10
175/175 [==============================] - 16s 93ms/step - loss: 0.7689 - accuracy: 0.6986 - val_loss: 0.2547 - val_accurac

In [125]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

4
Epoch 1/10
219/219 [==============================] - 20s 89ms/step - loss: 0.8741 - accuracy: 0.6513 - val_loss: 0.2437 - val_accuracy: 0.9125
Epoch 2/10
219/219 [==============================] - 20s 89ms/step - loss: 0.8180 - accuracy: 0.6753 - val_loss: 0.2480 - val_accuracy: 0.9121
Epoch 3/10
219/219 [==============================] - 20s 90ms/step - loss: 0.7971 - accuracy: 0.6825 - val_loss: 0.2394 - val_accuracy: 0.9152
Epoch 4/10
219/219 [==============================] - 20s 89ms/step - loss: 0.7685 - accuracy: 0.6990 - val_loss: 0.2424 - val_accuracy: 0.9151
Epoch 5/10
219/219 [==============================] - 20s 90ms/step - loss: 0.7528 - accuracy: 0.7098 - val_loss: 0.2401 - val_accuracy: 0.9176
Epoch 6/10
219/219 [==============================] - 20s 90ms/step - loss: 0.7315 - accuracy: 0.7178 - val_loss: 0.2537 - val_accuracy: 0.9138
Epoch 7/10
219/219 [==============================] - 20s 89ms/step - loss: 0.7229 - accuracy: 0.7235 - val_loss: 0.2718 - val_accurac

In [126]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

5
Epoch 1/10
263/263 [==============================] - 24s 87ms/step - loss: 0.8079 - accuracy: 0.6894 - val_loss: 0.2335 - val_accuracy: 0.9170
Epoch 2/10
263/263 [==============================] - 23s 87ms/step - loss: 0.7553 - accuracy: 0.7092 - val_loss: 0.2446 - val_accuracy: 0.9137
Epoch 3/10
263/263 [==============================] - 23s 87ms/step - loss: 0.7265 - accuracy: 0.7244 - val_loss: 0.2364 - val_accuracy: 0.9188
Epoch 4/10
263/263 [==============================] - 23s 87ms/step - loss: 0.7142 - accuracy: 0.7321 - val_loss: 0.2606 - val_accuracy: 0.9079
Epoch 5/10
263/263 [==============================] - 23s 87ms/step - loss: 0.6933 - accuracy: 0.7418 - val_loss: 0.2423 - val_accuracy: 0.9151
Epoch 6/10
263/263 [==============================] - 23s 87ms/step - loss: 0.6745 - accuracy: 0.7482 - val_loss: 0.2439 - val_accuracy: 0.9181
Epoch 7/10
263/263 [==============================] - 23s 87ms/step - loss: 0.6543 - accuracy: 0.7565 - val_loss: 0.2583 - val_accurac

In [127]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

6
Epoch 1/10
307/307 [==============================] - 27s 84ms/step - loss: 0.7403 - accuracy: 0.7213 - val_loss: 0.2466 - val_accuracy: 0.9131
Epoch 2/10
307/307 [==============================] - 26s 84ms/step - loss: 0.6848 - accuracy: 0.7475 - val_loss: 0.2450 - val_accuracy: 0.9143
Epoch 3/10
307/307 [==============================] - 26s 84ms/step - loss: 0.6810 - accuracy: 0.7494 - val_loss: 0.2438 - val_accuracy: 0.9156
Epoch 4/10
307/307 [==============================] - 26s 85ms/step - loss: 0.6651 - accuracy: 0.7549 - val_loss: 0.2442 - val_accuracy: 0.9165
Epoch 5/10
307/307 [==============================] - 26s 84ms/step - loss: 0.6464 - accuracy: 0.7632 - val_loss: 0.2483 - val_accuracy: 0.9171
Epoch 6/10
307/307 [==============================] - 26s 84ms/step - loss: 0.6287 - accuracy: 0.7726 - val_loss: 0.2693 - val_accuracy: 0.9039
Epoch 7/10
307/307 [==============================] - 26s 85ms/step - loss: 0.6334 - accuracy: 0.7733 - val_loss: 0.2924 - val_accurac

In [128]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

7
Epoch 1/10
350/350 [==============================] - 29s 82ms/step - loss: 0.6865 - accuracy: 0.7515 - val_loss: 0.2408 - val_accuracy: 0.9146
Epoch 2/10
350/350 [==============================] - 28s 81ms/step - loss: 0.6432 - accuracy: 0.7675 - val_loss: 0.2384 - val_accuracy: 0.9169
Epoch 3/10
350/350 [==============================] - 29s 82ms/step - loss: 0.6268 - accuracy: 0.7758 - val_loss: 0.2509 - val_accuracy: 0.9152
Epoch 4/10
350/350 [==============================] - 29s 82ms/step - loss: 0.6049 - accuracy: 0.7839 - val_loss: 0.2504 - val_accuracy: 0.9124
Epoch 5/10
350/350 [==============================] - 29s 82ms/step - loss: 0.6026 - accuracy: 0.7882 - val_loss: 0.2520 - val_accuracy: 0.9143
Epoch 6/10
350/350 [==============================] - 29s 82ms/step - loss: 0.5985 - accuracy: 0.7910 - val_loss: 0.2515 - val_accuracy: 0.9169
Epoch 7/10
350/350 [==============================] - 29s 83ms/step - loss: 0.5901 - accuracy: 0.7964 - val_loss: 0.2446 - val_accurac

In [129]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

8
Epoch 1/10
394/394 [==============================] - 33s 82ms/step - loss: 0.6181 - accuracy: 0.7796 - val_loss: 0.2373 - val_accuracy: 0.9166
Epoch 2/10
394/394 [==============================] - 32s 82ms/step - loss: 0.6026 - accuracy: 0.7883 - val_loss: 0.2342 - val_accuracy: 0.9178
Epoch 3/10
394/394 [==============================] - 34s 87ms/step - loss: 0.5855 - accuracy: 0.7962 - val_loss: 0.2471 - val_accuracy: 0.9177
Epoch 4/10
394/394 [==============================] - 32s 82ms/step - loss: 0.5778 - accuracy: 0.7991 - val_loss: 0.2477 - val_accuracy: 0.9115
Epoch 5/10
394/394 [==============================] - 33s 84ms/step - loss: 0.5628 - accuracy: 0.8046 - val_loss: 0.2373 - val_accuracy: 0.9201
Epoch 6/10
394/394 [==============================] - 32s 82ms/step - loss: 0.5644 - accuracy: 0.8061 - val_loss: 0.2497 - val_accuracy: 0.9154
Epoch 7/10
394/394 [==============================] - 32s 82ms/step - loss: 0.5684 - accuracy: 0.8115 - val_loss: 0.2458 - val_accurac

In [130]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

9
Epoch 1/10
438/438 [==============================] - 36s 81ms/step - loss: 0.5796 - accuracy: 0.7974 - val_loss: 0.2494 - val_accuracy: 0.9111
Epoch 2/10
438/438 [==============================] - 36s 81ms/step - loss: 0.5592 - accuracy: 0.8053 - val_loss: 0.2570 - val_accuracy: 0.9094
Epoch 3/10
438/438 [==============================] - 35s 81ms/step - loss: 0.5366 - accuracy: 0.8139 - val_loss: 0.2475 - val_accuracy: 0.9157
Epoch 4/10
438/438 [==============================] - 36s 82ms/step - loss: 0.5391 - accuracy: 0.8182 - val_loss: 0.2445 - val_accuracy: 0.9160
Epoch 5/10
438/438 [==============================] - 36s 82ms/step - loss: 0.5480 - accuracy: 0.8154 - val_loss: 0.2828 - val_accuracy: 0.9089
Epoch 6/10
438/438 [==============================] - 36s 82ms/step - loss: 0.5438 - accuracy: 0.8202 - val_loss: 0.2619 - val_accuracy: 0.9153
Epoch 7/10
438/438 [==============================] - 36s 82ms/step - loss: 0.5432 - accuracy: 0.8228 - val_loss: 0.2880 - val_accurac

In [131]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

10
Epoch 1/10
482/482 [==============================] - 42s 85ms/step - loss: 0.5304 - accuracy: 0.8152 - val_loss: 0.2383 - val_accuracy: 0.9147
Epoch 2/10
482/482 [==============================] - 40s 84ms/step - loss: 0.5171 - accuracy: 0.8215 - val_loss: 0.2721 - val_accuracy: 0.9090
Epoch 3/10
482/482 [==============================] - 40s 84ms/step - loss: 0.5107 - accuracy: 0.8258 - val_loss: 0.3002 - val_accuracy: 0.9074
Epoch 4/10
482/482 [==============================] - 40s 83ms/step - loss: 0.5092 - accuracy: 0.8283 - val_loss: 0.2534 - val_accuracy: 0.9111
Epoch 5/10
482/482 [==============================] - 40s 84ms/step - loss: 0.5151 - accuracy: 0.8265 - val_loss: 0.2434 - val_accuracy: 0.9153
Epoch 6/10
482/482 [==============================] - 40s 83ms/step - loss: 0.5157 - accuracy: 0.8309 - val_loss: 0.2723 - val_accuracy: 0.9083
Epoch 7/10
482/482 [==============================] - 40s 84ms/step - loss: 0.5198 - accuracy: 0.8313 - val_loss: 0.2713 - val_accura

In [132]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

11
Epoch 1/10
525/525 [==============================] - 44s 82ms/step - loss: 0.4964 - accuracy: 0.8279 - val_loss: 0.2420 - val_accuracy: 0.9126
Epoch 2/10
525/525 [==============================] - 43s 83ms/step - loss: 0.4836 - accuracy: 0.8344 - val_loss: 0.2467 - val_accuracy: 0.9136
Epoch 3/10
525/525 [==============================] - 42s 81ms/step - loss: 0.4846 - accuracy: 0.8364 - val_loss: 0.2413 - val_accuracy: 0.9143
Epoch 4/10
525/525 [==============================] - 42s 80ms/step - loss: 0.4822 - accuracy: 0.8398 - val_loss: 0.2432 - val_accuracy: 0.9163
Epoch 5/10
525/525 [==============================] - 42s 80ms/step - loss: 0.4840 - accuracy: 0.8417 - val_loss: 0.2680 - val_accuracy: 0.9070
Epoch 6/10
525/525 [==============================] - 42s 80ms/step - loss: 0.4945 - accuracy: 0.8399 - val_loss: 0.2473 - val_accuracy: 0.9165
Epoch 7/10
525/525 [==============================] - 43s 81ms/step - loss: 0.5003 - accuracy: 0.8411 - val_loss: 0.2735 - val_accura

In [133]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

12
Epoch 1/10
569/569 [==============================] - 45s 78ms/step - loss: 0.4650 - accuracy: 0.8388 - val_loss: 0.2390 - val_accuracy: 0.9173
Epoch 2/10
569/569 [==============================] - 44s 78ms/step - loss: 0.4474 - accuracy: 0.8448 - val_loss: 0.2483 - val_accuracy: 0.9164
Epoch 3/10
569/569 [==============================] - 45s 79ms/step - loss: 0.4593 - accuracy: 0.8474 - val_loss: 0.2450 - val_accuracy: 0.9150
Epoch 4/10
569/569 [==============================] - 44s 78ms/step - loss: 0.4588 - accuracy: 0.8497 - val_loss: 0.2578 - val_accuracy: 0.9136
Epoch 5/10
569/569 [==============================] - 44s 78ms/step - loss: 0.4660 - accuracy: 0.8490 - val_loss: 0.3039 - val_accuracy: 0.9119
Epoch 6/10
569/569 [==============================] - 44s 78ms/step - loss: 0.4725 - accuracy: 0.8486 - val_loss: 0.2630 - val_accuracy: 0.9120
Epoch 7/10
569/569 [==============================] - 44s 78ms/step - loss: 0.4808 - accuracy: 0.8470 - val_loss: 0.2679 - val_accura

In [134]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

13
Epoch 1/10
613/613 [==============================] - 48s 77ms/step - loss: 0.4342 - accuracy: 0.8507 - val_loss: 0.2584 - val_accuracy: 0.9079
Epoch 2/10
613/613 [==============================] - 48s 78ms/step - loss: 0.4317 - accuracy: 0.8560 - val_loss: 0.2492 - val_accuracy: 0.9116
Epoch 3/10
613/613 [==============================] - 47s 77ms/step - loss: 0.4365 - accuracy: 0.8570 - val_loss: 0.2531 - val_accuracy: 0.9116
Epoch 4/10
613/613 [==============================] - 47s 77ms/step - loss: 0.4407 - accuracy: 0.8558 - val_loss: 0.2493 - val_accuracy: 0.9127
Epoch 5/10
613/613 [==============================] - 47s 77ms/step - loss: 0.4476 - accuracy: 0.8540 - val_loss: 0.2752 - val_accuracy: 0.9079
Epoch 6/10
613/613 [==============================] - 47s 77ms/step - loss: 0.4631 - accuracy: 0.8532 - val_loss: 0.2799 - val_accuracy: 0.9112
Epoch 7/10
613/613 [==============================] - 47s 77ms/step - loss: 0.4743 - accuracy: 0.8481 - val_loss: 0.2967 - val_accura

In [135]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

14
Epoch 1/10
657/657 [==============================] - 52s 77ms/step - loss: 0.4104 - accuracy: 0.8603 - val_loss: 0.2466 - val_accuracy: 0.9136
Epoch 2/10
657/657 [==============================] - 50s 77ms/step - loss: 0.4061 - accuracy: 0.8642 - val_loss: 0.2537 - val_accuracy: 0.9132
Epoch 3/10
657/657 [==============================] - 51s 77ms/step - loss: 0.4115 - accuracy: 0.8640 - val_loss: 0.2595 - val_accuracy: 0.9121
Epoch 4/10
657/657 [==============================] - 50s 77ms/step - loss: 0.4212 - accuracy: 0.8635 - val_loss: 0.2601 - val_accuracy: 0.9134
Epoch 5/10
657/657 [==============================] - 50s 77ms/step - loss: 0.4333 - accuracy: 0.8612 - val_loss: 0.3240 - val_accuracy: 0.9091
Epoch 6/10
657/657 [==============================] - 50s 77ms/step - loss: 0.4451 - accuracy: 0.8614 - val_loss: 0.4137 - val_accuracy: 0.9065
Epoch 7/10
657/657 [==============================] - 50s 77ms/step - loss: 0.4513 - accuracy: 0.8583 - val_loss: 0.2813 - val_accura

In [136]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

15
Epoch 1/10
700/700 [==============================] - 54s 75ms/step - loss: 0.3899 - accuracy: 0.8662 - val_loss: 0.2587 - val_accuracy: 0.9076
Epoch 2/10
700/700 [==============================] - 53s 75ms/step - loss: 0.3816 - accuracy: 0.8721 - val_loss: 0.2598 - val_accuracy: 0.9164
Epoch 3/10
700/700 [==============================] - 53s 75ms/step - loss: 0.3879 - accuracy: 0.8709 - val_loss: 0.2603 - val_accuracy: 0.9122
Epoch 4/10
700/700 [==============================] - 53s 75ms/step - loss: 0.4086 - accuracy: 0.8695 - val_loss: 0.2643 - val_accuracy: 0.9098
Epoch 5/10
700/700 [==============================] - 53s 75ms/step - loss: 0.4195 - accuracy: 0.8677 - val_loss: 0.2586 - val_accuracy: 0.9104
Epoch 6/10
700/700 [==============================] - 53s 76ms/step - loss: 0.4352 - accuracy: 0.8630 - val_loss: 0.2875 - val_accuracy: 0.9041
Epoch 7/10
700/700 [==============================] - 53s 75ms/step - loss: 0.4377 - accuracy: 0.8634 - val_loss: 0.2855 - val_accura

In [137]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

16
Epoch 1/10
744/744 [==============================] - 58s 76ms/step - loss: 0.3695 - accuracy: 0.8745 - val_loss: 0.2403 - val_accuracy: 0.9186
Epoch 2/10
744/744 [==============================] - 57s 76ms/step - loss: 0.3672 - accuracy: 0.8771 - val_loss: 0.2498 - val_accuracy: 0.9139
Epoch 3/10
744/744 [==============================] - 57s 76ms/step - loss: 0.3774 - accuracy: 0.8757 - val_loss: 0.2614 - val_accuracy: 0.9129
Epoch 4/10
744/744 [==============================] - 58s 79ms/step - loss: 0.3925 - accuracy: 0.8733 - val_loss: 0.2687 - val_accuracy: 0.9112
Epoch 5/10
744/744 [==============================] - 57s 77ms/step - loss: 0.4003 - accuracy: 0.8744 - val_loss: 0.2642 - val_accuracy: 0.9094
Epoch 6/10
744/744 [==============================] - 58s 78ms/step - loss: 0.4252 - accuracy: 0.8684 - val_loss: 0.3205 - val_accuracy: 0.8981
Epoch 7/10
744/744 [==============================] - 57s 77ms/step - loss: 0.4357 - accuracy: 0.8658 - val_loss: 0.3026 - val_accura

In [138]:
time.sleep(400)

In [139]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

17
Epoch 1/10
788/788 [==============================] - 55s 69ms/step - loss: 0.3546 - accuracy: 0.8799 - val_loss: 0.2481 - val_accuracy: 0.9131
Epoch 2/10
788/788 [==============================] - 54s 69ms/step - loss: 0.3605 - accuracy: 0.8811 - val_loss: 0.2612 - val_accuracy: 0.9122
Epoch 3/10
788/788 [==============================] - 55s 69ms/step - loss: 0.3672 - accuracy: 0.8819 - val_loss: 0.3065 - val_accuracy: 0.9099
Epoch 4/10
788/788 [==============================] - 54s 69ms/step - loss: 0.3762 - accuracy: 0.8806 - val_loss: 0.2848 - val_accuracy: 0.9074
Epoch 5/10
788/788 [==============================] - 54s 69ms/step - loss: 0.3969 - accuracy: 0.8759 - val_loss: 0.2700 - val_accuracy: 0.9116
Epoch 6/10
788/788 [==============================] - 54s 69ms/step - loss: 0.4041 - accuracy: 0.8757 - val_loss: 0.3307 - val_accuracy: 0.9066
Epoch 7/10
788/788 [==============================] - 54s 69ms/step - loss: 0.4229 - accuracy: 0.8698 - val_loss: 0.3182 - val_accura

In [140]:
time.sleep(400)

In [141]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

18
Epoch 1/10
832/832 [==============================] - 58s 69ms/step - loss: 0.3329 - accuracy: 0.8877 - val_loss: 0.2499 - val_accuracy: 0.9136
Epoch 2/10
832/832 [==============================] - 58s 69ms/step - loss: 0.3421 - accuracy: 0.8882 - val_loss: 0.2738 - val_accuracy: 0.9107
Epoch 3/10
832/832 [==============================] - 58s 69ms/step - loss: 0.3537 - accuracy: 0.8868 - val_loss: 0.2879 - val_accuracy: 0.9080
Epoch 4/10
832/832 [==============================] - 58s 69ms/step - loss: 0.3661 - accuracy: 0.8823 - val_loss: 0.2969 - val_accuracy: 0.9074
Epoch 5/10
832/832 [==============================] - 58s 70ms/step - loss: 0.3871 - accuracy: 0.8795 - val_loss: 0.2792 - val_accuracy: 0.9061
Epoch 6/10
832/832 [==============================] - 58s 70ms/step - loss: 0.3971 - accuracy: 0.8761 - val_loss: 0.3187 - val_accuracy: 0.9006
Epoch 7/10
832/832 [==============================] - 58s 69ms/step - loss: 0.4104 - accuracy: 0.8742 - val_loss: 0.3294 - val_accura

In [142]:
time.sleep(400)

In [143]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

19
Epoch 1/10
875/875 [==============================] - 62s 70ms/step - loss: 0.3266 - accuracy: 0.8907 - val_loss: 0.2558 - val_accuracy: 0.9136
Epoch 2/10
875/875 [==============================] - 61s 69ms/step - loss: 0.3250 - accuracy: 0.8941 - val_loss: 0.2625 - val_accuracy: 0.9076
Epoch 3/10
875/875 [==============================] - 60s 69ms/step - loss: 0.3422 - accuracy: 0.8913 - val_loss: 0.2634 - val_accuracy: 0.9118
Epoch 4/10
875/875 [==============================] - 53s 61ms/step - loss: 0.3569 - accuracy: 0.8867 - val_loss: 0.2929 - val_accuracy: 0.9074
Epoch 5/10
875/875 [==============================] - 54s 61ms/step - loss: 0.3675 - accuracy: 0.8847 - val_loss: 0.3252 - val_accuracy: 0.8921
Epoch 6/10
875/875 [==============================] - 54s 61ms/step - loss: 0.3846 - accuracy: 0.8813 - val_loss: 0.2993 - val_accuracy: 0.8974
Epoch 7/10
875/875 [==============================] - 53s 61ms/step - loss: 0.3948 - accuracy: 0.8778 - val_loss: 0.3210 - val_accura

In [144]:
len(models_se)

20

In [145]:
new_model_se_dir  = "D:/models/aug_22/"+dataset+"/C2/"+dataset+"_model_c2_may_se_e1"

i=0

for model in models_se:
    model.save(new_model_se_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_se_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_se_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_se_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_se_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_se_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_se_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_se_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_se_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/mo

In [146]:
try:
    del se_values
    del top_images_by_se
    del top_labels_by_se
    del image_sets_se
    del label_sets_se
    del models_se
except:
    print("Error")

In [147]:
gc.collect()

349489

## Training guided by Random values

In [148]:
import random
random_indexes =list(range(len(x_train_and_adversary)))
random.shuffle(random_indexes)
print(random_indexes[:10])
print(len(random_indexes))

[44452, 19510, 17221, 10503, 39361, 13991, 33222, 18588, 30257, 15715]
55998


In [149]:
save_dir = "D:/guided-retraining/data/"+dataset+"/random_values.npy"
random_indexes = np.load(save_dir)

In [150]:
len(random_indexes)

55998

In [151]:
# Obtaining top n images by random values
top_images_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),x_train_and_adversary)
top_labels_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),y_train_and_adversary)

In [152]:
m = n_data_points
n = 0
image_sets_random = []
label_sets_random = []



for i in range((len(top_images_by_random)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_random)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_random)%m))
        top_images_by_random_n = np.array(top_images_by_random[:n+m+(len(top_images_by_random)%m)])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m+(len(top_images_by_random)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_random_n = np.array(top_images_by_random[:n+m])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m])
    image_sets_random.append(top_images_by_random_n)
    label_sets_random.append(top_labels_by_random_n)
    print(len(top_images_by_random_n))
    n += m




0 :
0  ->  2800
2800
1 :
0  ->  5600
5600
2 :
0  ->  8400
8400
3 :
0  ->  11200
11200
4 :
0  ->  14000
14000
5 :
0  ->  16800
16800
6 :
0  ->  19600
19600
7 :
0  ->  22400
22400
8 :
0  ->  25200
25200
9 :
0  ->  28000
28000
10 :
0  ->  30800
30800
11 :
0  ->  33600
33600
12 :
0  ->  36400
36400
13 :
0  ->  39200
39200
14 :
0  ->  42000
42000
15 :
0  ->  44800
44800
16 :
0  ->  47600
47600
17 :
0  ->  50400
50400
18 :
0  ->  53200
53200
19 :
Last
0  ->  58798
55998


In [153]:
len(image_sets_random)

20

In [167]:
print(model_dir)

models_random = []
for i in range(len(label_sets_random)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_random.append(model)


D:/guided-retraining/models/model_fashion_2
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [155]:
n=0
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

0
Epoch 1/10
44/44 [==============================] - 6s 131ms/step - loss: 0.3010 - accuracy: 0.8946 - val_loss: 0.2403 - val_accuracy: 0.9171
Epoch 2/10
44/44 [==============================] - 6s 127ms/step - loss: 0.2678 - accuracy: 0.9093 - val_loss: 0.2423 - val_accuracy: 0.9150
Epoch 3/10
44/44 [==============================] - 6s 129ms/step - loss: 0.2683 - accuracy: 0.9025 - val_loss: 0.2458 - val_accuracy: 0.9174
Epoch 4/10
44/44 [==============================] - 6s 128ms/step - loss: 0.2358 - accuracy: 0.9179 - val_loss: 0.2643 - val_accuracy: 0.9131
Epoch 5/10
44/44 [==============================] - 6s 129ms/step - loss: 0.2280 - accuracy: 0.9186 - val_loss: 0.2739 - val_accuracy: 0.9100
Epoch 6/10
44/44 [==============================] - 6s 131ms/step - loss: 0.2181 - accuracy: 0.9218 - val_loss: 0.2800 - val_accuracy: 0.9143
Epoch 7/10
44/44 [==============================] - 6s 135ms/step - loss: 0.2043 - accuracy: 0.9250 - val_loss: 0.2901 - val_accuracy: 0.9123
Epoc

In [156]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

1
Epoch 1/10
88/88 [==============================] - 10s 105ms/step - loss: 0.3309 - accuracy: 0.8852 - val_loss: 0.2488 - val_accuracy: 0.9102
Epoch 2/10
88/88 [==============================] - 9s 99ms/step - loss: 0.3052 - accuracy: 0.8948 - val_loss: 0.2397 - val_accuracy: 0.9181
Epoch 3/10
88/88 [==============================] - 9s 99ms/step - loss: 0.2876 - accuracy: 0.9018 - val_loss: 0.2488 - val_accuracy: 0.9159
Epoch 4/10
88/88 [==============================] - 9s 100ms/step - loss: 0.2771 - accuracy: 0.9011 - val_loss: 0.2505 - val_accuracy: 0.9146
Epoch 5/10
88/88 [==============================] - 9s 99ms/step - loss: 0.2549 - accuracy: 0.9102 - val_loss: 0.2741 - val_accuracy: 0.9096
Epoch 6/10
88/88 [==============================] - 9s 100ms/step - loss: 0.2446 - accuracy: 0.9125 - val_loss: 0.2748 - val_accuracy: 0.9134
Epoch 7/10
88/88 [==============================] - 9s 100ms/step - loss: 0.2442 - accuracy: 0.9089 - val_loss: 0.2783 - val_accuracy: 0.9099
Epoch 

In [157]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

2
Epoch 1/10
132/132 [==============================] - 12s 89ms/step - loss: 0.3198 - accuracy: 0.8912 - val_loss: 0.2439 - val_accuracy: 0.9135
Epoch 2/10
132/132 [==============================] - 12s 90ms/step - loss: 0.3023 - accuracy: 0.8994 - val_loss: 0.2446 - val_accuracy: 0.9161
Epoch 3/10
132/132 [==============================] - 12s 92ms/step - loss: 0.2960 - accuracy: 0.8962 - val_loss: 0.2442 - val_accuracy: 0.9149
Epoch 4/10
132/132 [==============================] - 12s 91ms/step - loss: 0.2956 - accuracy: 0.8993 - val_loss: 0.3503 - val_accuracy: 0.8849
Epoch 5/10
132/132 [==============================] - 12s 90ms/step - loss: 0.2722 - accuracy: 0.9071 - val_loss: 0.2595 - val_accuracy: 0.9111
Epoch 6/10
132/132 [==============================] - 12s 90ms/step - loss: 0.2745 - accuracy: 0.9050 - val_loss: 0.2733 - val_accuracy: 0.9115
Epoch 7/10
132/132 [==============================] - 12s 90ms/step - loss: 0.2642 - accuracy: 0.9110 - val_loss: 0.2733 - val_accurac

In [158]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

3
Epoch 1/10
175/175 [==============================] - 15s 83ms/step - loss: 0.3241 - accuracy: 0.8923 - val_loss: 0.2440 - val_accuracy: 0.9131
Epoch 2/10
175/175 [==============================] - 15s 83ms/step - loss: 0.3042 - accuracy: 0.8977 - val_loss: 0.2404 - val_accuracy: 0.9176
Epoch 3/10
175/175 [==============================] - 15s 84ms/step - loss: 0.2948 - accuracy: 0.9010 - val_loss: 0.2590 - val_accuracy: 0.9129
Epoch 4/10
175/175 [==============================] - 15s 84ms/step - loss: 0.2895 - accuracy: 0.9034 - val_loss: 0.2587 - val_accuracy: 0.9134
Epoch 5/10
175/175 [==============================] - 14s 83ms/step - loss: 0.2846 - accuracy: 0.9058 - val_loss: 0.2487 - val_accuracy: 0.9151
Epoch 6/10
175/175 [==============================] - 15s 84ms/step - loss: 0.2781 - accuracy: 0.9062 - val_loss: 0.2856 - val_accuracy: 0.9099
Epoch 7/10
175/175 [==============================] - 15s 83ms/step - loss: 0.2727 - accuracy: 0.9093 - val_loss: 0.3021 - val_accurac

In [159]:
print(n)#
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

4
Epoch 1/10
219/219 [==============================] - 19s 80ms/step - loss: 0.3139 - accuracy: 0.8930 - val_loss: 0.2513 - val_accuracy: 0.9108
Epoch 2/10
219/219 [==============================] - 18s 81ms/step - loss: 0.2950 - accuracy: 0.8980 - val_loss: 0.2654 - val_accuracy: 0.9119
Epoch 3/10
219/219 [==============================] - 18s 82ms/step - loss: 0.2948 - accuracy: 0.9001 - val_loss: 0.2693 - val_accuracy: 0.9130
Epoch 4/10
219/219 [==============================] - 18s 81ms/step - loss: 0.2838 - accuracy: 0.9056 - val_loss: 0.2594 - val_accuracy: 0.9166
Epoch 5/10
219/219 [==============================] - 18s 81ms/step - loss: 0.2866 - accuracy: 0.9027 - val_loss: 0.2878 - val_accuracy: 0.9121
Epoch 6/10
219/219 [==============================] - 18s 81ms/step - loss: 0.2911 - accuracy: 0.9059 - val_loss: 0.2728 - val_accuracy: 0.9136
Epoch 7/10
219/219 [==============================] - 18s 83ms/step - loss: 0.2872 - accuracy: 0.9045 - val_loss: 0.2694 - val_accurac

In [160]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

5
Epoch 1/10
263/263 [==============================] - 21s 78ms/step - loss: 0.3154 - accuracy: 0.8905 - val_loss: 0.2468 - val_accuracy: 0.9115
Epoch 2/10
263/263 [==============================] - 20s 78ms/step - loss: 0.3062 - accuracy: 0.8971 - val_loss: 0.2581 - val_accuracy: 0.9114
Epoch 3/10
263/263 [==============================] - 20s 78ms/step - loss: 0.2958 - accuracy: 0.9008 - val_loss: 0.2798 - val_accuracy: 0.9089
Epoch 4/10
263/263 [==============================] - 20s 78ms/step - loss: 0.2963 - accuracy: 0.9001 - val_loss: 0.2585 - val_accuracy: 0.9162
Epoch 5/10
263/263 [==============================] - 20s 77ms/step - loss: 0.2980 - accuracy: 0.9019 - val_loss: 0.2758 - val_accuracy: 0.9099
Epoch 6/10
263/263 [==============================] - 21s 78ms/step - loss: 0.2952 - accuracy: 0.9007 - val_loss: 0.2633 - val_accuracy: 0.9101
Epoch 7/10
263/263 [==============================] - 20s 77ms/step - loss: 0.3024 - accuracy: 0.9010 - val_loss: 0.2637 - val_accurac

In [161]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

6
Epoch 1/10
307/307 [==============================] - 24s 75ms/step - loss: 0.3181 - accuracy: 0.8935 - val_loss: 0.2612 - val_accuracy: 0.9101
Epoch 2/10
307/307 [==============================] - 23s 75ms/step - loss: 0.3086 - accuracy: 0.8948 - val_loss: 0.2497 - val_accuracy: 0.9154
Epoch 3/10
307/307 [==============================] - 23s 76ms/step - loss: 0.3084 - accuracy: 0.8968 - val_loss: 0.2577 - val_accuracy: 0.9159
Epoch 4/10
307/307 [==============================] - 23s 76ms/step - loss: 0.3044 - accuracy: 0.8986 - val_loss: 0.2666 - val_accuracy: 0.9121
Epoch 5/10
307/307 [==============================] - 23s 75ms/step - loss: 0.2958 - accuracy: 0.8990 - val_loss: 0.2868 - val_accuracy: 0.9076
Epoch 6/10
307/307 [==============================] - 23s 76ms/step - loss: 0.2944 - accuracy: 0.9010 - val_loss: 0.2715 - val_accuracy: 0.9136
Epoch 7/10
307/307 [==============================] - 23s 75ms/step - loss: 0.3003 - accuracy: 0.9019 - val_loss: 0.3260 - val_accurac

In [162]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

7
Epoch 1/10
350/350 [==============================] - 26s 74ms/step - loss: 0.3133 - accuracy: 0.8940 - val_loss: 0.2433 - val_accuracy: 0.9161
Epoch 2/10
350/350 [==============================] - 26s 74ms/step - loss: 0.3039 - accuracy: 0.8956 - val_loss: 0.2530 - val_accuracy: 0.9141
Epoch 3/10
350/350 [==============================] - 30s 87ms/step - loss: 0.3000 - accuracy: 0.9001 - val_loss: 0.2574 - val_accuracy: 0.9146
Epoch 4/10
350/350 [==============================] - 29s 83ms/step - loss: 0.3044 - accuracy: 0.9010 - val_loss: 0.2775 - val_accuracy: 0.9071
Epoch 5/10
350/350 [==============================] - 27s 77ms/step - loss: 0.2991 - accuracy: 0.9024 - val_loss: 0.2782 - val_accuracy: 0.9097
Epoch 6/10
350/350 [==============================] - 27s 77ms/step - loss: 0.3002 - accuracy: 0.9010 - val_loss: 0.2900 - val_accuracy: 0.9089
Epoch 7/10
350/350 [==============================] - 25s 73ms/step - loss: 0.3134 - accuracy: 0.9008 - val_loss: 0.2953 - val_accurac

In [163]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

8
Epoch 1/10
394/394 [==============================] - 29s 71ms/step - loss: 0.3210 - accuracy: 0.8942 - val_loss: 0.2557 - val_accuracy: 0.9118
Epoch 2/10
394/394 [==============================] - 28s 72ms/step - loss: 0.3079 - accuracy: 0.8958 - val_loss: 0.2610 - val_accuracy: 0.9097
Epoch 3/10
394/394 [==============================] - 29s 73ms/step - loss: 0.3086 - accuracy: 0.8971 - val_loss: 0.2626 - val_accuracy: 0.9113
Epoch 4/10
394/394 [==============================] - 31s 79ms/step - loss: 0.3068 - accuracy: 0.8987 - val_loss: 0.2909 - val_accuracy: 0.9132
Epoch 5/10
394/394 [==============================] - 32s 82ms/step - loss: 0.3149 - accuracy: 0.8995 - val_loss: 0.2822 - val_accuracy: 0.9105
Epoch 6/10
394/394 [==============================] - 33s 83ms/step - loss: 0.3164 - accuracy: 0.8976 - val_loss: 0.2765 - val_accuracy: 0.9069
Epoch 7/10
394/394 [==============================] - 32s 82ms/step - loss: 0.3268 - accuracy: 0.8942 - val_loss: 0.2758 - val_accurac

In [168]:
print(models_random[8].evaluate(x_test_and_adversary,y_test_and_adversary))
print(models_random[9].evaluate(x_test_and_adversary,y_test_and_adversary))

438/438 [==============================] - 5s 10ms/step - loss: 0.5032 - accuracy: 0.8342
[0.5032416582107544, 0.8342142701148987]
438/438 [==============================] - 5s 10ms/step - loss: 0.5032 - accuracy: 0.8342
[0.5032416582107544, 0.8342142701148987]


In [170]:

n= 9
print(n)#
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

9
Epoch 1/10
438/438 [==============================] - 36s 80ms/step - loss: 0.3228 - accuracy: 0.8929 - val_loss: 0.2447 - val_accuracy: 0.9129
Epoch 2/10
438/438 [==============================] - 40s 92ms/step - loss: 0.3096 - accuracy: 0.8940 - val_loss: 0.2623 - val_accuracy: 0.9137
Epoch 3/10
438/438 [==============================] - 38s 86ms/step - loss: 0.3155 - accuracy: 0.8957 - val_loss: 0.2646 - val_accuracy: 0.9073
Epoch 4/10
438/438 [==============================] - 34s 77ms/step - loss: 0.3120 - accuracy: 0.8980 - val_loss: 0.2599 - val_accuracy: 0.9074
Epoch 5/10
438/438 [==============================] - 34s 79ms/step - loss: 0.3224 - accuracy: 0.8949 - val_loss: 0.2623 - val_accuracy: 0.9100
Epoch 6/10
438/438 [==============================] - 34s 78ms/step - loss: 0.3258 - accuracy: 0.8956 - val_loss: 0.2954 - val_accuracy: 0.8979
Epoch 7/10
438/438 [==============================] - 34s 77ms/step - loss: 0.3319 - accuracy: 0.8970 - val_loss: 0.2829 - val_accurac

In [171]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

10
Epoch 1/10
482/482 [==============================] - 38s 77ms/step - loss: 0.3202 - accuracy: 0.8938 - val_loss: 0.2509 - val_accuracy: 0.9128
Epoch 2/10
482/482 [==============================] - 39s 82ms/step - loss: 0.3079 - accuracy: 0.8969 - val_loss: 0.2539 - val_accuracy: 0.9116
Epoch 3/10
482/482 [==============================] - 37s 78ms/step - loss: 0.3152 - accuracy: 0.8981 - val_loss: 0.2671 - val_accuracy: 0.9074
Epoch 4/10
482/482 [==============================] - 39s 80ms/step - loss: 0.3192 - accuracy: 0.8965 - val_loss: 0.2653 - val_accuracy: 0.9078
Epoch 5/10
482/482 [==============================] - 41s 85ms/step - loss: 0.3229 - accuracy: 0.8973 - val_loss: 0.2655 - val_accuracy: 0.9038
Epoch 6/10
482/482 [==============================] - 39s 82ms/step - loss: 0.3287 - accuracy: 0.8937 - val_loss: 0.2725 - val_accuracy: 0.9105
Epoch 7/10
482/482 [==============================] - 42s 86ms/step - loss: 0.3332 - accuracy: 0.8941 - val_loss: 0.3185 - val_accura

In [172]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

11
Epoch 1/10
525/525 [==============================] - 38s 71ms/step - loss: 0.3203 - accuracy: 0.8917 - val_loss: 0.2651 - val_accuracy: 0.9124
Epoch 2/10
525/525 [==============================] - 44s 83ms/step - loss: 0.3148 - accuracy: 0.8964 - val_loss: 0.2668 - val_accuracy: 0.9063
Epoch 3/10
525/525 [==============================] - 43s 82ms/step - loss: 0.3189 - accuracy: 0.8960 - val_loss: 0.2534 - val_accuracy: 0.9142
Epoch 4/10
525/525 [==============================] - 40s 76ms/step - loss: 0.3251 - accuracy: 0.8960 - val_loss: 0.2534 - val_accuracy: 0.9156
Epoch 5/10
525/525 [==============================] - 45s 85ms/step - loss: 0.3262 - accuracy: 0.8955 - val_loss: 0.3231 - val_accuracy: 0.8940
Epoch 6/10
525/525 [==============================] - 41s 79ms/step - loss: 0.3441 - accuracy: 0.8913 - val_loss: 0.2912 - val_accuracy: 0.9013
Epoch 7/10
525/525 [==============================] - 39s 75ms/step - loss: 0.3456 - accuracy: 0.8915 - val_loss: 0.3084 - val_accura

In [173]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

12
Epoch 1/10
569/569 [==============================] - 44s 76ms/step - loss: 0.3189 - accuracy: 0.8936 - val_loss: 0.2450 - val_accuracy: 0.9124
Epoch 2/10
569/569 [==============================] - 43s 76ms/step - loss: 0.3185 - accuracy: 0.8958 - val_loss: 0.2481 - val_accuracy: 0.9133
Epoch 3/10
569/569 [==============================] - 43s 75ms/step - loss: 0.3149 - accuracy: 0.8969 - val_loss: 0.2871 - val_accuracy: 0.9069
Epoch 4/10
569/569 [==============================] - 44s 77ms/step - loss: 0.3336 - accuracy: 0.8926 - val_loss: 0.2711 - val_accuracy: 0.9091
Epoch 5/10
569/569 [==============================] - 46s 80ms/step - loss: 0.3374 - accuracy: 0.8930 - val_loss: 0.2779 - val_accuracy: 0.9104
Epoch 6/10
569/569 [==============================] - 46s 80ms/step - loss: 0.3492 - accuracy: 0.8880 - val_loss: 0.2885 - val_accuracy: 0.9069
Epoch 7/10
569/569 [==============================] - 45s 78ms/step - loss: 0.3529 - accuracy: 0.8896 - val_loss: 0.2769 - val_accura

In [174]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

13
Epoch 1/10
613/613 [==============================] - 52s 83ms/step - loss: 0.3156 - accuracy: 0.8943 - val_loss: 0.2586 - val_accuracy: 0.9119
Epoch 2/10
613/613 [==============================] - 54s 88ms/step - loss: 0.3199 - accuracy: 0.8957 - val_loss: 0.2469 - val_accuracy: 0.9164
Epoch 3/10
613/613 [==============================] - 55s 90ms/step - loss: 0.3243 - accuracy: 0.8940 - val_loss: 0.3015 - val_accuracy: 0.9079
Epoch 4/10
613/613 [==============================] - 50s 82ms/step - loss: 0.3339 - accuracy: 0.8928 - val_loss: 0.2756 - val_accuracy: 0.9105
Epoch 5/10
613/613 [==============================] - 50s 81ms/step - loss: 0.3401 - accuracy: 0.8923 - val_loss: 0.2860 - val_accuracy: 0.9049
Epoch 6/10
613/613 [==============================] - 50s 81ms/step - loss: 0.3523 - accuracy: 0.8871 - val_loss: 0.2916 - val_accuracy: 0.9011
Epoch 7/10
613/613 [==============================] - 50s 81ms/step - loss: 0.3663 - accuracy: 0.8861 - val_loss: 0.2858 - val_accura

In [175]:
print(n)#
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

14
Epoch 1/10
657/657 [==============================] - 59s 88ms/step - loss: 0.3202 - accuracy: 0.8927 - val_loss: 0.2551 - val_accuracy: 0.9109
Epoch 2/10
657/657 [==============================] - 54s 83ms/step - loss: 0.3164 - accuracy: 0.8952 - val_loss: 0.2537 - val_accuracy: 0.9134
Epoch 3/10
657/657 [==============================] - 56s 85ms/step - loss: 0.3279 - accuracy: 0.8943 - val_loss: 0.3021 - val_accuracy: 0.9113
Epoch 4/10
657/657 [==============================] - 52s 79ms/step - loss: 0.3351 - accuracy: 0.8931 - val_loss: 0.3220 - val_accuracy: 0.9021
Epoch 5/10
657/657 [==============================] - 56s 85ms/step - loss: 0.3531 - accuracy: 0.8893 - val_loss: 0.3248 - val_accuracy: 0.8970
Epoch 6/10
657/657 [==============================] - 53s 80ms/step - loss: 0.3571 - accuracy: 0.8872 - val_loss: 0.3865 - val_accuracy: 0.8859
Epoch 7/10
657/657 [==============================] - 54s 82ms/step - loss: 0.3679 - accuracy: 0.8869 - val_loss: 0.2994 - val_accura

In [176]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

15
Epoch 1/10
700/700 [==============================] - 58s 82ms/step - loss: 0.3207 - accuracy: 0.8913 - val_loss: 0.2553 - val_accuracy: 0.9143
Epoch 2/10
700/700 [==============================] - 57s 81ms/step - loss: 0.3173 - accuracy: 0.8944 - val_loss: 0.2580 - val_accuracy: 0.9110
Epoch 3/10
700/700 [==============================] - 64s 91ms/step - loss: 0.3280 - accuracy: 0.8941 - val_loss: 0.2608 - val_accuracy: 0.9082
Epoch 4/10
700/700 [==============================] - 57s 81ms/step - loss: 0.3442 - accuracy: 0.8897 - val_loss: 0.2627 - val_accuracy: 0.9105
Epoch 5/10
700/700 [==============================] - 57s 82ms/step - loss: 0.3556 - accuracy: 0.8896 - val_loss: 0.3186 - val_accuracy: 0.9012
Epoch 6/10
700/700 [==============================] - 58s 83ms/step - loss: 0.3633 - accuracy: 0.8855 - val_loss: 0.2805 - val_accuracy: 0.9061
Epoch 7/10
700/700 [==============================] - 55s 78ms/step - loss: 0.3757 - accuracy: 0.8854 - val_loss: 0.3982 - val_accura

In [177]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

16
Epoch 1/10
744/744 [==============================] - 66s 87ms/step - loss: 0.3239 - accuracy: 0.8918 - val_loss: 0.2519 - val_accuracy: 0.9117
Epoch 2/10
744/744 [==============================] - 67s 90ms/step - loss: 0.3247 - accuracy: 0.8941 - val_loss: 0.2764 - val_accuracy: 0.9016
Epoch 3/10
744/744 [==============================] - 63s 84ms/step - loss: 0.3374 - accuracy: 0.8916 - val_loss: 0.2833 - val_accuracy: 0.9126
Epoch 4/10
744/744 [==============================] - 62s 83ms/step - loss: 0.3580 - accuracy: 0.8878 - val_loss: 0.2736 - val_accuracy: 0.9091
Epoch 5/10
744/744 [==============================] - 61s 82ms/step - loss: 0.3644 - accuracy: 0.8843 - val_loss: 0.2977 - val_accuracy: 0.9032
Epoch 6/10
744/744 [==============================] - 60s 80ms/step - loss: 0.3755 - accuracy: 0.8828 - val_loss: 0.4185 - val_accuracy: 0.8840
Epoch 7/10
744/744 [==============================] - 58s 78ms/step - loss: 0.3841 - accuracy: 0.8814 - val_loss: 0.3150 - val_accura

In [ ]:
time.sleep(400)

In [178]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

17
Epoch 1/10
788/788 [==============================] - 66s 83ms/step - loss: 0.3180 - accuracy: 0.8930 - val_loss: 0.2506 - val_accuracy: 0.9108
Epoch 2/10
788/788 [==============================] - 63s 80ms/step - loss: 0.3150 - accuracy: 0.8949 - val_loss: 0.2651 - val_accuracy: 0.9069
Epoch 3/10
788/788 [==============================] - 63s 81ms/step - loss: 0.3297 - accuracy: 0.8933 - val_loss: 0.3274 - val_accuracy: 0.9026
Epoch 4/10
788/788 [==============================] - 63s 80ms/step - loss: 0.3495 - accuracy: 0.8903 - val_loss: 0.2832 - val_accuracy: 0.9056
Epoch 5/10
788/788 [==============================] - 63s 80ms/step - loss: 0.3596 - accuracy: 0.8870 - val_loss: 0.3139 - val_accuracy: 0.9030
Epoch 6/10
788/788 [==============================] - 63s 80ms/step - loss: 0.3772 - accuracy: 0.8832 - val_loss: 0.3476 - val_accuracy: 0.8951
Epoch 7/10
788/788 [==============================] - 65s 83ms/step - loss: 0.3928 - accuracy: 0.8788 - val_loss: 0.4219 - val_accura

In [ ]:
time.sleep(400)

In [179]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

18
Epoch 1/10
832/832 [==============================] - 74s 88ms/step - loss: 0.3209 - accuracy: 0.8919 - val_loss: 0.2616 - val_accuracy: 0.9117
Epoch 2/10
832/832 [==============================] - 73s 87ms/step - loss: 0.3261 - accuracy: 0.8924 - val_loss: 0.2888 - val_accuracy: 0.9069
Epoch 3/10
832/832 [==============================] - 71s 85ms/step - loss: 0.3413 - accuracy: 0.8919 - val_loss: 0.2856 - val_accuracy: 0.9079
Epoch 4/10
832/832 [==============================] - 76s 92ms/step - loss: 0.3585 - accuracy: 0.8861 - val_loss: 0.2799 - val_accuracy: 0.9044
Epoch 5/10
832/832 [==============================] - 67s 80ms/step - loss: 0.3737 - accuracy: 0.8839 - val_loss: 0.3153 - val_accuracy: 0.9021
Epoch 6/10
832/832 [==============================] - 68s 82ms/step - loss: 0.3865 - accuracy: 0.8828 - val_loss: 0.3248 - val_accuracy: 0.8991
Epoch 7/10
832/832 [==============================] - 66s 79ms/step - loss: 0.3894 - accuracy: 0.8811 - val_loss: 0.3002 - val_accura

In [ ]:
time.sleep(400)

In [180]:
print(n)#
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

19
Epoch 1/10
875/875 [==============================] - 66s 74ms/step - loss: 0.3190 - accuracy: 0.8912 - val_loss: 0.2520 - val_accuracy: 0.9142
Epoch 2/10
875/875 [==============================] - 66s 75ms/step - loss: 0.3315 - accuracy: 0.8920 - val_loss: 0.2555 - val_accuracy: 0.9111
Epoch 3/10
875/875 [==============================] - 66s 75ms/step - loss: 0.3492 - accuracy: 0.8890 - val_loss: 0.2761 - val_accuracy: 0.9023
Epoch 4/10
875/875 [==============================] - 67s 76ms/step - loss: 0.3679 - accuracy: 0.8842 - val_loss: 0.2843 - val_accuracy: 0.9046
Epoch 5/10
875/875 [==============================] - 69s 78ms/step - loss: 0.3776 - accuracy: 0.8812 - val_loss: 0.3042 - val_accuracy: 0.8953
Epoch 6/10
875/875 [==============================] - 68s 78ms/step - loss: 0.3971 - accuracy: 0.8781 - val_loss: 0.2942 - val_accuracy: 0.8988
Epoch 7/10
875/875 [==============================] - 66s 76ms/step - loss: 0.4087 - accuracy: 0.8750 - val_loss: 0.3514 - val_accura

In [181]:
new_model_random_dir  = "D:/models/aug_22/"+dataset+"/C2/"+dataset+"_model_c2_may_random_e1"

i=9

for model in models_random[i:]:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_random_e1_9\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_random_e1_10\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_random_e1_11\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_random_e1_12\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_random_e1_13\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_random_e1_14\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_random_e1_15\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_random_e1_16\assets
Model has been saved
I

In [ ]:
loading = False

models_random = []

if loading:
    for i in range(20):
        model_random_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_random_e1_"+str(i)
        print(model_random_dir)
        model =utils.My_model('gtsrb',True,model_random_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_random.append(model)
       

In [182]:
try:
    del random_indexes
    del top_images_by_random
    del top_labels_by_random
    del image_sets_random
    del label_sets_random
    del models_random
except:
    print("Error")

In [183]:
gc.collect()

391929

## Training guided by NC

In [17]:
# NC
nc_values = []
for i in range(1,17):
    #save_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/NC_values/nc_values_"+str(i)+".npy"
    save_dir = "D:/guided-retraining/data/"+dataset+"/"+dataset+"_nc_values_"+str(i)+".npy"

    #print(save_dir_rand)
    tmp_values = np.load(save_dir)
    #print(tmp_values.shape)
    nc_values = np.append(nc_values,tmp_values)

In [18]:
nc_values.shape

(55998,)

In [19]:

top_images_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),x_train_and_adversary)
top_labels_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),y_train_and_adversary)


In [20]:
m = n_data_points
n = 0
image_sets_nc = []
label_sets_nc = []


for i in range((len(top_images_by_nc)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_nc)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_nc)%m))
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m+(len(top_images_by_nc)%m)])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m+(len(top_images_by_nc)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m])
    image_sets_nc.append(top_images_by_nc_n)
    label_sets_nc.append(top_labels_by_nc_n)
    print(len(top_images_by_nc_n))
    n += m




0 :
0  ->  2800
2800
1 :
0  ->  5600
5600
2 :
0  ->  8400
8400
3 :
0  ->  11200
11200
4 :
0  ->  14000
14000
5 :
0  ->  16800
16800
6 :
0  ->  19600
19600
7 :
0  ->  22400
22400
8 :
0  ->  25200
25200
9 :
0  ->  28000
28000
10 :
0  ->  30800
30800
11 :
0  ->  33600
33600
12 :
0  ->  36400
36400
13 :
0  ->  39200
39200
14 :
0  ->  42000
42000
15 :
0  ->  44800
44800
16 :
0  ->  47600
47600
17 :
0  ->  50400
50400
18 :
0  ->  53200
53200
19 :
Last
0  ->  58798
55998


In [21]:
print(model_dir)

models_nc = []
for i in range(len(label_sets_nc)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_nc.append(model)


D:/guided-retraining/models/model_fashion_2
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [22]:
n=0
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

0
Epoch 1/10
44/44 [==============================] - 5s 110ms/step - loss: 0.3872 - accuracy: 0.8886 - val_loss: 0.3002 - val_accuracy: 0.8904
Epoch 2/10
44/44 [==============================] - 5s 112ms/step - loss: 0.3243 - accuracy: 0.9061 - val_loss: 0.2902 - val_accuracy: 0.8955
Epoch 3/10
44/44 [==============================] - 5s 123ms/step - loss: 0.3043 - accuracy: 0.9071 - val_loss: 0.2837 - val_accuracy: 0.8971
Epoch 4/10
44/44 [==============================] - 6s 131ms/step - loss: 0.2802 - accuracy: 0.9154 - val_loss: 0.3441 - val_accuracy: 0.8861
Epoch 5/10
44/44 [==============================] - 5s 126ms/step - loss: 0.2750 - accuracy: 0.9118 - val_loss: 0.3213 - val_accuracy: 0.8914
Epoch 6/10
44/44 [==============================] - 5s 118ms/step - loss: 0.2492 - accuracy: 0.9250 - val_loss: 0.3245 - val_accuracy: 0.8937
Epoch 7/10
44/44 [==============================] - 5s 116ms/step - loss: 0.2461 - accuracy: 0.9225 - val_loss: 0.3359 - val_accuracy: 0.8923
Epoc

In [23]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

1
Epoch 1/10
88/88 [==============================] - 8s 89ms/step - loss: 0.3348 - accuracy: 0.9120 - val_loss: 0.2864 - val_accuracy: 0.8974
Epoch 2/10
88/88 [==============================] - 8s 95ms/step - loss: 0.2816 - accuracy: 0.9186 - val_loss: 0.2990 - val_accuracy: 0.8955
Epoch 3/10
88/88 [==============================] - 9s 106ms/step - loss: 0.2660 - accuracy: 0.9229 - val_loss: 0.3424 - val_accuracy: 0.8890
Epoch 4/10
88/88 [==============================] - 8s 96ms/step - loss: 0.2508 - accuracy: 0.9237 - val_loss: 0.3335 - val_accuracy: 0.8969
Epoch 5/10
88/88 [==============================] - 9s 105ms/step - loss: 0.2378 - accuracy: 0.9325 - val_loss: 0.3845 - val_accuracy: 0.8795
Epoch 6/10
88/88 [==============================] - 9s 102ms/step - loss: 0.2360 - accuracy: 0.9314 - val_loss: 0.3877 - val_accuracy: 0.8884
Epoch 7/10
88/88 [==============================] - 8s 96ms/step - loss: 0.2197 - accuracy: 0.9325 - val_loss: 0.4198 - val_accuracy: 0.8810
Epoch 8/

In [24]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

2
Epoch 1/10
132/132 [==============================] - 11s 80ms/step - loss: 0.2471 - accuracy: 0.9336 - val_loss: 0.3089 - val_accuracy: 0.8984
Epoch 2/10
132/132 [==============================] - 11s 80ms/step - loss: 0.2327 - accuracy: 0.9386 - val_loss: 0.4270 - val_accuracy: 0.8822
Epoch 3/10
132/132 [==============================] - 10s 77ms/step - loss: 0.2255 - accuracy: 0.9388 - val_loss: 0.3819 - val_accuracy: 0.8901
Epoch 4/10
132/132 [==============================] - 10s 78ms/step - loss: 0.2061 - accuracy: 0.9433 - val_loss: 0.3392 - val_accuracy: 0.8937
Epoch 5/10
132/132 [==============================] - 10s 77ms/step - loss: 0.2079 - accuracy: 0.9402 - val_loss: 0.4260 - val_accuracy: 0.8824
Epoch 6/10
132/132 [==============================] - 10s 73ms/step - loss: 0.1929 - accuracy: 0.9469 - val_loss: 0.4710 - val_accuracy: 0.8770
Epoch 7/10
132/132 [==============================] - 10s 72ms/step - loss: 0.1902 - accuracy: 0.9482 - val_loss: 0.5135 - val_accurac

In [25]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

3
Epoch 1/10
175/175 [==============================] - 13s 69ms/step - loss: 0.2461 - accuracy: 0.9334 - val_loss: 0.3136 - val_accuracy: 0.8971
Epoch 2/10
175/175 [==============================] - 12s 67ms/step - loss: 0.2160 - accuracy: 0.9399 - val_loss: 0.3211 - val_accuracy: 0.8988
Epoch 3/10
175/175 [==============================] - 12s 67ms/step - loss: 0.2024 - accuracy: 0.9425 - val_loss: 0.3448 - val_accuracy: 0.8996
Epoch 4/10
175/175 [==============================] - 12s 67ms/step - loss: 0.2000 - accuracy: 0.9444 - val_loss: 0.4421 - val_accuracy: 0.8824
Epoch 5/10
175/175 [==============================] - 12s 68ms/step - loss: 0.1981 - accuracy: 0.9451 - val_loss: 0.4775 - val_accuracy: 0.8812
Epoch 6/10
175/175 [==============================] - 12s 69ms/step - loss: 0.1904 - accuracy: 0.9438 - val_loss: 0.5032 - val_accuracy: 0.8777
Epoch 7/10
175/175 [==============================] - 12s 68ms/step - loss: 0.1888 - accuracy: 0.9461 - val_loss: 0.5660 - val_accurac

In [26]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

4
Epoch 1/10
219/219 [==============================] - 15s 67ms/step - loss: 0.2345 - accuracy: 0.9378 - val_loss: 0.2932 - val_accuracy: 0.9001
Epoch 2/10
219/219 [==============================] - 15s 68ms/step - loss: 0.2043 - accuracy: 0.9436 - val_loss: 0.3473 - val_accuracy: 0.8963
Epoch 3/10
219/219 [==============================] - 16s 75ms/step - loss: 0.2045 - accuracy: 0.9451 - val_loss: 0.4696 - val_accuracy: 0.8845
Epoch 4/10
219/219 [==============================] - 15s 67ms/step - loss: 0.1941 - accuracy: 0.9462 - val_loss: 0.4672 - val_accuracy: 0.8746
Epoch 5/10
219/219 [==============================] - 15s 67ms/step - loss: 0.1941 - accuracy: 0.9466 - val_loss: 0.5975 - val_accuracy: 0.8489
Epoch 6/10
219/219 [==============================] - 14s 66ms/step - loss: 0.1945 - accuracy: 0.9461 - val_loss: 0.6841 - val_accuracy: 0.8239
Epoch 7/10
219/219 [==============================] - 15s 67ms/step - loss: 0.1914 - accuracy: 0.9491 - val_loss: 0.8476 - val_accurac

In [27]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

5
Epoch 1/10
263/263 [==============================] - 21s 76ms/step - loss: 0.2237 - accuracy: 0.9424 - val_loss: 0.3290 - val_accuracy: 0.9003
Epoch 2/10
263/263 [==============================] - 24s 91ms/step - loss: 0.2034 - accuracy: 0.9458 - val_loss: 0.3362 - val_accuracy: 0.8928
Epoch 3/10
263/263 [==============================] - 19s 72ms/step - loss: 0.1956 - accuracy: 0.9449 - val_loss: 0.4391 - val_accuracy: 0.8682
Epoch 4/10
263/263 [==============================] - 21s 79ms/step - loss: 0.1970 - accuracy: 0.9471 - val_loss: 0.8643 - val_accuracy: 0.7854
Epoch 5/10
263/263 [==============================] - 19s 72ms/step - loss: 0.1961 - accuracy: 0.9470 - val_loss: 0.8440 - val_accuracy: 0.7850
Epoch 6/10
263/263 [==============================] - 19s 71ms/step - loss: 0.1886 - accuracy: 0.9496 - val_loss: 0.9370 - val_accuracy: 0.7832
Epoch 7/10
263/263 [==============================] - 19s 71ms/step - loss: 0.1898 - accuracy: 0.9505 - val_loss: 1.5838 - val_accurac

In [28]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

6
Epoch 1/10
307/307 [==============================] - 24s 77ms/step - loss: 0.2024 - accuracy: 0.9476 - val_loss: 0.3649 - val_accuracy: 0.8851
Epoch 2/10
307/307 [==============================] - 22s 73ms/step - loss: 0.1944 - accuracy: 0.9491 - val_loss: 0.4432 - val_accuracy: 0.8697
Epoch 3/10
307/307 [==============================] - 21s 70ms/step - loss: 0.1946 - accuracy: 0.9494 - val_loss: 0.5508 - val_accuracy: 0.8497
Epoch 4/10
307/307 [==============================] - 22s 71ms/step - loss: 0.1941 - accuracy: 0.9504 - val_loss: 0.9640 - val_accuracy: 0.7954
Epoch 5/10
307/307 [==============================] - 22s 71ms/step - loss: 0.1944 - accuracy: 0.9512 - val_loss: 1.0716 - val_accuracy: 0.7836
Epoch 6/10
307/307 [==============================] - 22s 72ms/step - loss: 0.1982 - accuracy: 0.9501 - val_loss: 1.4798 - val_accuracy: 0.7723
Epoch 7/10
307/307 [==============================] - 22s 73ms/step - loss: 0.1935 - accuracy: 0.9519 - val_loss: 1.7683 - val_accurac

In [29]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

7
Epoch 1/10
350/350 [==============================] - 25s 71ms/step - loss: 0.1948 - accuracy: 0.9482 - val_loss: 0.3444 - val_accuracy: 0.8976
Epoch 2/10
350/350 [==============================] - 25s 71ms/step - loss: 0.1877 - accuracy: 0.9523 - val_loss: 0.4269 - val_accuracy: 0.8810
Epoch 3/10
350/350 [==============================] - 25s 70ms/step - loss: 0.1840 - accuracy: 0.9509 - val_loss: 0.4689 - val_accuracy: 0.8416
Epoch 4/10
350/350 [==============================] - 24s 69ms/step - loss: 0.1802 - accuracy: 0.9521 - val_loss: 0.8158 - val_accuracy: 0.7910
Epoch 5/10
350/350 [==============================] - 24s 69ms/step - loss: 0.1886 - accuracy: 0.9498 - val_loss: 1.2974 - val_accuracy: 0.7884
Epoch 6/10
350/350 [==============================] - 25s 70ms/step - loss: 0.1857 - accuracy: 0.9524 - val_loss: 1.2054 - val_accuracy: 0.7895
Epoch 7/10
350/350 [==============================] - 27s 78ms/step - loss: 0.1908 - accuracy: 0.9529 - val_loss: 0.9889 - val_accurac

In [30]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

8
Epoch 1/10
394/394 [==============================] - 34s 86ms/step - loss: 0.1956 - accuracy: 0.9494 - val_loss: 0.3461 - val_accuracy: 0.8969
Epoch 2/10
394/394 [==============================] - 31s 78ms/step - loss: 0.1838 - accuracy: 0.9511 - val_loss: 0.4200 - val_accuracy: 0.8918
Epoch 3/10
394/394 [==============================] - 30s 77ms/step - loss: 0.1879 - accuracy: 0.9509 - val_loss: 0.4354 - val_accuracy: 0.8872
Epoch 4/10
394/394 [==============================] - 29s 75ms/step - loss: 0.1809 - accuracy: 0.9519 - val_loss: 0.5839 - val_accuracy: 0.8603
Epoch 5/10
394/394 [==============================] - 29s 73ms/step - loss: 0.1823 - accuracy: 0.9517 - val_loss: 1.1329 - val_accuracy: 0.7897
Epoch 6/10
394/394 [==============================] - 28s 72ms/step - loss: 0.1916 - accuracy: 0.9513 - val_loss: 1.0067 - val_accuracy: 0.7863
Epoch 7/10
394/394 [==============================] - 29s 73ms/step - loss: 0.1976 - accuracy: 0.9511 - val_loss: 1.0952 - val_accurac

In [31]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

9
Epoch 1/10
438/438 [==============================] - 35s 77ms/step - loss: 0.1912 - accuracy: 0.9494 - val_loss: 0.3156 - val_accuracy: 0.8968
Epoch 2/10
438/438 [==============================] - 33s 74ms/step - loss: 0.1838 - accuracy: 0.9515 - val_loss: 0.4424 - val_accuracy: 0.8874
Epoch 3/10
438/438 [==============================] - 30s 69ms/step - loss: 0.1887 - accuracy: 0.9503 - val_loss: 0.3999 - val_accuracy: 0.8929
Epoch 4/10
438/438 [==============================] - 34s 77ms/step - loss: 0.1824 - accuracy: 0.9492 - val_loss: 0.4081 - val_accuracy: 0.8879
Epoch 5/10
438/438 [==============================] - 33s 75ms/step - loss: 0.1950 - accuracy: 0.9484 - val_loss: 0.4385 - val_accuracy: 0.8798
Epoch 6/10
438/438 [==============================] - 30s 69ms/step - loss: 0.1992 - accuracy: 0.9505 - val_loss: 0.4851 - val_accuracy: 0.8760
Epoch 7/10
438/438 [==============================] - 31s 72ms/step - loss: 0.2090 - accuracy: 0.9473 - val_loss: 0.5091 - val_accurac

In [32]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

10
Epoch 1/10
482/482 [==============================] - 36s 73ms/step - loss: 0.2028 - accuracy: 0.9471 - val_loss: 0.3475 - val_accuracy: 0.8991
Epoch 2/10
482/482 [==============================] - 34s 70ms/step - loss: 0.1933 - accuracy: 0.9498 - val_loss: 0.4069 - val_accuracy: 0.8899
Epoch 3/10
482/482 [==============================] - 32s 67ms/step - loss: 0.1873 - accuracy: 0.9494 - val_loss: 0.3845 - val_accuracy: 0.8954
Epoch 4/10
482/482 [==============================] - 33s 68ms/step - loss: 0.1940 - accuracy: 0.9497 - val_loss: 0.3908 - val_accuracy: 0.8915
Epoch 5/10
482/482 [==============================] - 33s 68ms/step - loss: 0.1966 - accuracy: 0.9498 - val_loss: 0.4248 - val_accuracy: 0.8804
Epoch 6/10
482/482 [==============================] - 33s 68ms/step - loss: 0.2074 - accuracy: 0.9463 - val_loss: 0.4409 - val_accuracy: 0.8814
Epoch 7/10
482/482 [==============================] - 39s 82ms/step - loss: 0.2045 - accuracy: 0.9475 - val_loss: 0.4053 - val_accura

In [33]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

11
Epoch 1/10
525/525 [==============================] - 36s 68ms/step - loss: 0.2016 - accuracy: 0.9461 - val_loss: 0.3267 - val_accuracy: 0.8986
Epoch 2/10
525/525 [==============================] - 35s 66ms/step - loss: 0.1955 - accuracy: 0.9482 - val_loss: 0.3703 - val_accuracy: 0.8973
Epoch 3/10
525/525 [==============================] - 35s 66ms/step - loss: 0.2032 - accuracy: 0.9471 - val_loss: 0.4613 - val_accuracy: 0.8890
Epoch 4/10
525/525 [==============================] - 35s 66ms/step - loss: 0.2104 - accuracy: 0.9448 - val_loss: 0.3873 - val_accuracy: 0.8859
Epoch 5/10
525/525 [==============================] - 35s 66ms/step - loss: 0.2135 - accuracy: 0.9452 - val_loss: 0.3926 - val_accuracy: 0.8897
Epoch 6/10
525/525 [==============================] - 35s 67ms/step - loss: 0.2220 - accuracy: 0.9439 - val_loss: 0.6071 - val_accuracy: 0.8808
Epoch 7/10
525/525 [==============================] - 35s 66ms/step - loss: 0.2291 - accuracy: 0.9421 - val_loss: 0.4300 - val_accura

In [34]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

12
Epoch 1/10
569/569 [==============================] - 38s 66ms/step - loss: 0.2041 - accuracy: 0.9452 - val_loss: 0.3403 - val_accuracy: 0.9009
Epoch 2/10
569/569 [==============================] - 38s 66ms/step - loss: 0.1971 - accuracy: 0.9467 - val_loss: 0.3918 - val_accuracy: 0.8946
Epoch 3/10
569/569 [==============================] - 37s 65ms/step - loss: 0.1983 - accuracy: 0.9468 - val_loss: 0.5298 - val_accuracy: 0.8855
Epoch 4/10
569/569 [==============================] - 37s 66ms/step - loss: 0.2064 - accuracy: 0.9456 - val_loss: 0.3998 - val_accuracy: 0.8974
Epoch 5/10
569/569 [==============================] - 37s 65ms/step - loss: 0.2111 - accuracy: 0.9460 - val_loss: 0.4708 - val_accuracy: 0.8920
Epoch 6/10
569/569 [==============================] - 37s 65ms/step - loss: 0.2241 - accuracy: 0.9434 - val_loss: 0.4739 - val_accuracy: 0.8889
Epoch 7/10
569/569 [==============================] - 38s 67ms/step - loss: 0.2306 - accuracy: 0.9420 - val_loss: 0.4328 - val_accura

In [35]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

13
Epoch 1/10
613/613 [==============================] - 42s 67ms/step - loss: 0.2007 - accuracy: 0.9447 - val_loss: 0.3259 - val_accuracy: 0.9007
Epoch 2/10
613/613 [==============================] - 40s 66ms/step - loss: 0.1995 - accuracy: 0.9456 - val_loss: 0.3592 - val_accuracy: 0.8944
Epoch 3/10
613/613 [==============================] - 41s 67ms/step - loss: 0.2047 - accuracy: 0.9459 - val_loss: 0.3175 - val_accuracy: 0.8936
Epoch 4/10
613/613 [==============================] - 41s 66ms/step - loss: 0.2095 - accuracy: 0.9456 - val_loss: 0.3492 - val_accuracy: 0.8949
Epoch 5/10
613/613 [==============================] - 40s 65ms/step - loss: 0.2194 - accuracy: 0.9435 - val_loss: 0.4462 - val_accuracy: 0.8911
Epoch 6/10
613/613 [==============================] - 40s 65ms/step - loss: 0.2240 - accuracy: 0.9429 - val_loss: 0.3945 - val_accuracy: 0.8921
Epoch 7/10
613/613 [==============================] - 40s 65ms/step - loss: 0.2340 - accuracy: 0.9420 - val_loss: 0.6076 - val_accura

In [36]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

14
Epoch 1/10
657/657 [==============================] - 45s 67ms/step - loss: 0.2079 - accuracy: 0.9439 - val_loss: 0.3486 - val_accuracy: 0.8963
Epoch 2/10
657/657 [==============================] - 43s 66ms/step - loss: 0.2050 - accuracy: 0.9443 - val_loss: 0.3894 - val_accuracy: 0.9019
Epoch 3/10
657/657 [==============================] - 44s 66ms/step - loss: 0.2124 - accuracy: 0.9430 - val_loss: 0.3212 - val_accuracy: 0.8939
Epoch 4/10
657/657 [==============================] - 42s 64ms/step - loss: 0.2210 - accuracy: 0.9414 - val_loss: 0.3539 - val_accuracy: 0.8927
Epoch 5/10
657/657 [==============================] - 42s 64ms/step - loss: 0.2282 - accuracy: 0.9399 - val_loss: 0.4186 - val_accuracy: 0.8941
Epoch 6/10
657/657 [==============================] - 42s 65ms/step - loss: 0.2412 - accuracy: 0.9390 - val_loss: 0.5003 - val_accuracy: 0.8879
Epoch 7/10
657/657 [==============================] - 43s 65ms/step - loss: 0.2548 - accuracy: 0.9370 - val_loss: 0.3856 - val_accura

In [ ]:
time.sleep(400)

In [37]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

15
Epoch 1/10
700/700 [==============================] - 54s 77ms/step - loss: 0.2135 - accuracy: 0.9395 - val_loss: 0.3043 - val_accuracy: 0.9041
Epoch 2/10
700/700 [==============================] - 53s 76ms/step - loss: 0.2211 - accuracy: 0.9392 - val_loss: 0.3092 - val_accuracy: 0.8999
Epoch 3/10
700/700 [==============================] - 53s 76ms/step - loss: 0.2239 - accuracy: 0.9390 - val_loss: 0.3560 - val_accuracy: 0.9016
Epoch 4/10
700/700 [==============================] - 54s 77ms/step - loss: 0.2435 - accuracy: 0.9353 - val_loss: 0.3361 - val_accuracy: 0.8991
Epoch 5/10
700/700 [==============================] - 54s 77ms/step - loss: 0.2449 - accuracy: 0.9358 - val_loss: 0.4002 - val_accuracy: 0.8918
Epoch 6/10
700/700 [==============================] - 53s 76ms/step - loss: 0.2572 - accuracy: 0.9326 - val_loss: 0.4113 - val_accuracy: 0.8938
Epoch 7/10
700/700 [==============================] - 53s 76ms/step - loss: 0.2695 - accuracy: 0.9303 - val_loss: 0.3788 - val_accura

In [ ]:
time.sleep(400)

In [38]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

16
Epoch 1/10
744/744 [==============================] - 60s 80ms/step - loss: 0.2406 - accuracy: 0.9297 - val_loss: 0.2706 - val_accuracy: 0.9081
Epoch 2/10
744/744 [==============================] - 56s 76ms/step - loss: 0.2397 - accuracy: 0.9302 - val_loss: 0.2980 - val_accuracy: 0.9051
Epoch 3/10
744/744 [==============================] - 56s 75ms/step - loss: 0.2516 - accuracy: 0.9287 - val_loss: 0.3219 - val_accuracy: 0.9057
Epoch 4/10
744/744 [==============================] - 56s 75ms/step - loss: 0.2661 - accuracy: 0.9256 - val_loss: 0.3506 - val_accuracy: 0.8926
Epoch 5/10
744/744 [==============================] - 55s 75ms/step - loss: 0.2779 - accuracy: 0.9247 - val_loss: 0.3700 - val_accuracy: 0.8946
Epoch 6/10
744/744 [==============================] - 53s 71ms/step - loss: 0.2893 - accuracy: 0.9204 - val_loss: 0.4768 - val_accuracy: 0.8887
Epoch 7/10
744/744 [==============================] - 55s 74ms/step - loss: 0.2966 - accuracy: 0.9186 - val_loss: 0.4524 - val_accura

In [ ]:
time.sleep(400)

In [39]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

17
Epoch 1/10
788/788 [==============================] - 54s 67ms/step - loss: 0.2649 - accuracy: 0.9191 - val_loss: 0.2758 - val_accuracy: 0.9109
Epoch 2/10
788/788 [==============================] - 54s 69ms/step - loss: 0.2659 - accuracy: 0.9191 - val_loss: 0.3057 - val_accuracy: 0.9057
Epoch 3/10
788/788 [==============================] - 51s 64ms/step - loss: 0.2820 - accuracy: 0.9161 - val_loss: 0.2963 - val_accuracy: 0.9026
Epoch 4/10
788/788 [==============================] - 51s 64ms/step - loss: 0.2886 - accuracy: 0.9172 - val_loss: 0.4186 - val_accuracy: 0.8848
Epoch 5/10
788/788 [==============================] - 51s 64ms/step - loss: 0.3067 - accuracy: 0.9121 - val_loss: 0.3289 - val_accuracy: 0.8965
Epoch 6/10
788/788 [==============================] - 51s 65ms/step - loss: 0.3149 - accuracy: 0.9093 - val_loss: 0.3795 - val_accuracy: 0.8993
Epoch 7/10
788/788 [==============================] - 51s 65ms/step - loss: 0.3286 - accuracy: 0.9075 - val_loss: 0.3160 - val_accura

In [ ]:
time.sleep(400)

In [40]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

18
Epoch 1/10
832/832 [==============================] - 53s 63ms/step - loss: 0.2896 - accuracy: 0.9075 - val_loss: 0.2721 - val_accuracy: 0.9086
Epoch 2/10
832/832 [==============================] - 53s 64ms/step - loss: 0.2917 - accuracy: 0.9074 - val_loss: 0.2696 - val_accuracy: 0.9113
Epoch 3/10
832/832 [==============================] - 54s 65ms/step - loss: 0.3065 - accuracy: 0.9051 - val_loss: 0.3024 - val_accuracy: 0.9052
Epoch 4/10
832/832 [==============================] - 55s 66ms/step - loss: 0.3159 - accuracy: 0.9064 - val_loss: 0.3101 - val_accuracy: 0.9024
Epoch 5/10
832/832 [==============================] - 54s 65ms/step - loss: 0.3319 - accuracy: 0.8998 - val_loss: 0.2988 - val_accuracy: 0.9041
Epoch 6/10
832/832 [==============================] - 57s 69ms/step - loss: 0.3486 - accuracy: 0.8976 - val_loss: 0.3421 - val_accuracy: 0.8984
Epoch 7/10
832/832 [==============================] - 58s 70ms/step - loss: 0.3606 - accuracy: 0.8942 - val_loss: 0.3386 - val_accura

In [ ]:
time.sleep(400)

In [41]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

19
Epoch 1/10
875/875 [==============================] - 57s 64ms/step - loss: 0.3194 - accuracy: 0.8930 - val_loss: 0.2420 - val_accuracy: 0.9142
Epoch 2/10
875/875 [==============================] - 56s 63ms/step - loss: 0.3253 - accuracy: 0.8933 - val_loss: 0.2598 - val_accuracy: 0.9074
Epoch 3/10
875/875 [==============================] - 56s 64ms/step - loss: 0.3366 - accuracy: 0.8913 - val_loss: 0.2692 - val_accuracy: 0.9079
Epoch 4/10
875/875 [==============================] - 57s 65ms/step - loss: 0.3523 - accuracy: 0.8887 - val_loss: 0.2901 - val_accuracy: 0.9027
Epoch 5/10
875/875 [==============================] - 57s 65ms/step - loss: 0.3694 - accuracy: 0.8848 - val_loss: 0.2828 - val_accuracy: 0.9062
Epoch 6/10
875/875 [==============================] - 56s 63ms/step - loss: 0.3894 - accuracy: 0.8812 - val_loss: 0.3162 - val_accuracy: 0.8990
Epoch 7/10
875/875 [==============================] - 56s 64ms/step - loss: 0.3971 - accuracy: 0.8769 - val_loss: 0.3746 - val_accura

In [42]:
new_model_nc_dir  = "D:/models/aug_22/"+dataset+"/C2/"+dataset+"_model_c2_may_nc_e1"

i=0

for model in models_nc:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_nc_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_nc_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_nc_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_nc_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_nc_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_nc_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_nc_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/fashion/C2/fashion_model_c2_may_nc_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/mo

In [ ]:
loading = False

models_nc = []

if loading:
    for i in range(20):
        model_nc_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_nc_e1"+str(i)
        print(model_nc_dir)
        model =utils.My_model(dataset,True,model_nc_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_nc.append(model)
       

In [43]:
try:
    del nc_values
    del top_images_by_nc
    del top_labels_by_nc
    del image_sets_nc
    del label_sets_nc
    del models_nc
except:
    print("Error")

In [44]:
gc.collect()

327980